In [1]:
#!pip install json_repair numpy matplotlib tqdm pillow pymupdf

In [2]:
import os, base64, cv2, json_repair, requests, pymupdf
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image 
import pymupdf

In [3]:
erax_url_id = "a4li0qhs1a6f7e"
erax_url = f"https://api.runpod.ai/v2/{erax_url_id}/runsync"

erax_url_a100_id =  "jaeprlipf9ih6y"
erax_url_a100 =  f"https://api.runpod.ai/v2/{erax_url_a100_id}/runsync"

API_key_3P =  "API Key" # Digins

# Common prompts

In [4]:
sickness_medicines =  """
PHỤ LỤC I
(Ban hành kèm theo Thông tư số 05/2024/TT-BYT ngày 14 tháng 5 năm 2024 của Bộ trưởng Bộ Y tế)
I. Danh mục thuốc biệt dược gốc, sinh phẩm tham chiếu được áp dụng hình thức đàm phán giá
TT
Tên thuốc, sinh phẩm
Thành phần/hoạt chất
Nồng độ, hàm lượng
Dạng bào chế
Ghi chú
1
Aclasta
Acid zoledronic khan (tương ứng 5,33mg acid zoledronic monohydrate)
5mg/100ml
Dung dịch truyền tĩnh mạch
2
Actemra
Tocilizumab
20mg/ml
Dung dịch đậm đặc để pha dung dịch tiêm truyền
()
3
Actilyse
Alteplase
50mg
Bột đông khô và dung môi để pha dung dịch tiêm truyền
4
Adalat LA 30mg
Nifedipin
30mg
Viên nén phóng thích kéo dài
5
Advagraf
Tacrolimus (dưới dạng Tacrolimus monohydrate)
0,5mg
Viên nang cứng phóng thích kéo dài
6
Advagraf
Tacrolimus (dưới dạng Tacrolimus monohydrate)
1mg
Viên nang cứng phóng thích kéo dài
7
Advagraf
Tacrolimus (dưới dạng Tacrolimus monohydrate)
5mg
Viên nang cứng phóng thích kéo dài
8
Aerius
Desloratadine
5mg
Viên nén bao phim
9
Aerius
Desloratadine
0,5mg/ml
Siro
10
Afinitor 10mg
Everolimus
10mg
Viên nén
11
Afinitor 5mg
Everolimus
5mg
Viên nén
12
Alecensa
Alectinib (dưới dạng Alectinib hydrochloride)
150mg
Viên nang cứng
()
13
Alegysal
Pemirolast kali
5mg/5ml
Dung dịch nhỏ mắt
14
Alimta
Pemetrexed (dưới dạng pemetrexed dinatri heptahydrat)
500mg
Bột đông khô pha dung địch tiêm truyền
15 Alimta Pemetrexed (dưới dạng pemetrexed dinatri
100mg Bột đông khô pha dung dịch
heptahydrat)
tiêm truyền
16
Aloxi
Palonosetron (dưới dạng palonosetron hydroclorid)
0,25mg
Dung dịch tiêm truyền tĩnh mạch
17
Aminoplasmal® B. Braun 10% E
Isoleucine; Leucine; Lysine (dưới dạng Lysine hydrochloride); Methionine; Phenylalanine; Threonine; Tryptophan; Valine; Arginine; Histidine; Alanine; Glycine; Aspartic acid; Glutamic Acid; Proline; Serine; Tyrosine; Sodium acetate trihydrate; Sodium hydroxide; Potassium acetate; Magnesium chloride hexahydrate; Disodium phosphate dodecahydrate
Công thức bào chế tính cho một đơn vị đóng gói nhỏ nhất là chai 250ml: Isoleucine 1,25gam; Leucine 2,225gam; Lysine hydrochloride (tương đương với Lysine 1,7125 gam) 2,14 gam; Methionine 1,10gam; Phenylalanine 1,175gam; Threonine 1,05gam; Tryptophan 0,40gam; Valine 1,55gam; Arginine 2,875gam; Histidine 0,75gam; Alanine 2,625gam; Glycine 3,00gam; Aspartic acid 1,40gam; Glutamic Acid 1,80gam; Proline 1,375gam; Serine 0,575gam; Tyrosine 0,10gam; Sodium acetate trihydrate 0,7145gam; Sodium hydroxide 0,09gam; Potassium acetate 0,61325gam; Magnesium chloride hexahydrate 0,127gam; Disodium phosphate dodecahydrate 0,89525gam
Dung dịch tiêm truyền tĩnh mạch
18 Aminoplasmal® B.
Braun 5% E
Isoleucine; Leucine;
Lysine (dưới dạng Lysine hydrochloride);
Methionine;
Phenylalanine; Threonine;
Tryptophan; Valine;
Arginine; Histidine;
Công thức bào chế tính cho một đơn vị đóng gói nhỏ nhất là chai 250ml:
Isoleucine
0,625gam; Leucine
1,1125gam; Lysine
Dung dịch tiêm truyền tĩnh mạch
Alanine; Glycine; Aspartic acid; Glutamic Acid;
Proline; Serine; Tyrosine;
Sodium acetate trihydrate;
Sodium hydroxide;
Potassium acetate; Sodium chloride; Magnesium chloride hexahydrate;
Disodium phosphate dodecahydrate hydrochloride (tương đương với Lysine
0,8575 gam) 1,07 gam; Methionine
0,55 gam;
Phenylalanine
0,5875gam;
Threonine
0,525gam;
Tryptophan
0,20gam; Valine
0,775gam; Arginine
1,4375gam;
Histidine 0,375gam;
Alanine 1,3125gam;
Glycine 1,50gam;
Aspartic acid
0,70gam; Glutamic
Acid 0,9gam;
Proline 0,6875gam;
Serine 0,2875gam;
Tyrosine 0,10gam;
Sodium acetate trihydrate
0,34025gam;
Sodium hydroxide
0,035gam;
Potassium acetate
0,61325gam;
Sodium chloride
0,241 gam;
Magnesium chloride
hexahydrate 0,127gam; Disodium phosphate dodecahydrate 0,89525gam
19
Amlor
Amlodipine
5mg
Viên nang cứng
20
Amlor
Amlodipine
5mg
Viên nén
(*)
21
Anaropin
Ropivacain HCl (dưới dạng Ropivacain HCl monobydrat)
7,5mg/ml
Dung dịch tiêm/truyền
22
Anaropin
Ropivacain HCl (dưới dạng Ropivacain HCl monohydrat)
5mg/ml
Dung dịch tiêm/truyền
23
Anaropin
Ropivacain HCl (dưới dạng Ropivacain HCl monohydrat)
2mg/ml
Dung dịch tiêm/truyền
24 A 100 /16 7 l P li l 100 /16 7 l D dịh đậ
đặc pha dung dịch tiêm truyền
25
Anzatax 150mg/25ml
Paclitaxel
150mg/25ml
Dung dịch đậm đặc pha dung dịch tiêm truyền
26
Anzatax 30mg/5ml
Paclitaxel
30mg/5ml
Dung dịch đậm đặc pha dung dịch tiêm truyền
27
Apidra solostar
Insulin glulisine
300 đơn vị/3ml
Dung dịch tiêm trong bút tiêm nạp sẵn
28
Aprovel
Irbesartan
150mg
Viên nén bao phim
29
Aprovel
Irbesartan
300mg
Viên nén bao phim
30
Arcoxia 120mg
Etoricoxib
120mg
Viên nén bao phim
31
Arcoxia 60mg
Etoricoxib
60mg
Viên nén bao phim
32
Arcoxia 90mg
Etoricoxib
90mg
Viên nén bao phim
33
Arduan
Pipecuronium bromide
4mg
Bột đông khô pha tiêm
(*)
34
Arimidex
Anastrozole
1mg
Viên nén bao phim
35
Aromasin
Exemestane
25mg
Viên nén bao đường
36
Atelec Tablets 10
Cilnidipine
10mg
Viên nén bao phim
37
Augmentin 1g
Amoxicillin (dưới dạng Amoxicillin trihydrate); Acid clavulanic (dưới dạng Clavulanat potassium)
875mg; 125mg
Viên nén bao phim
38
Augmentin 250mg/31,25mg
Amoxicillin (dưới dạng Amoxicillin trihydrate); Acid clavulanic (dưới dạng Kali clavulanate)
250mg; 31,25mg
Bột pha hỗn dịch uống
39
Augmentin 500mg/62,5mg
Amoxicillin (dưới dạng Amoxicillin trihydrate); Acid clavulanic (dưới dạng Kali clavulanate)
500mg; 62,5mg
Bột pha hỗn dịch uống
40 Augmentin 625mg tablets
Amoxicillin (dưới dạng
Amoxicillin trihydrate);
500mg; 125mg Viên nén bao phim
Kali clavulanate)
41
Avamys
Fluticason Furoat
27,5mcg/liều xịt
Hỗn dịch xịt mũi
42
Avastin
Bevacizumab
100mg/4ml
Dung dịch đậm đặc để pha dung dịch tiêm truyền
43
Avastin
Bevacizumab
400mg/16ml
Dung dịch đậm đặc để pha dung dịch tiêm truyền
44
Avelox
Moxifloxacin (dưới dạng Moxifloxacin hydroclorid)
400mg/250ml
Dung dịch truyền tĩnh mạch
45
Avelox
Moxifloxacin (dưới dạng Moxifloxacin hydroclorid)
400mg
Viên nén bao phim
46
Avodart
Dutasteride
0,5mg
Viên nang mềm
47
Azopt
Brinzolamide
10mg/ml
Hỗn dịch nhỏ mắt
48
Bambec
Bambuterol hydroclorid
10mg
Viên nén
()
49
Berodual
Ipratropium bromide khan (dưới dạng Ipratropium bromide monohydrate); Fenoterol Hydrobromide
(0,02mg; 0,05mg)/nhát xịt
Dung dịch khí dung
50
Berodual
Ipratropium bromide khan; Fenoterol hydrobromide
250mcg/ml; 500mcg/ml
Dung dịch khí dung
()
51
Besivance
Besifloxacin (dưới dạng besifloxacin hydrochlorid)
0,6% (kl/tt)
Hỗn dịch nhỏ mắt
()
52
Betaloc 50mg
Metoprolol tartrate
50mg
Viên nén
()
53
Betaloc ZOK 25mg
Metoprolol succinat (tương đương với 25mg metoprolol tartrat hoặc 19,5mg metoprolol)
23,75mg
Viên nén phóng thích kéo dài
54
Betaloc Zok 50mg
Metoprolol succinat (tương đương với 50mg metoprolol tartrate hoặc 39mg metoprolol)
47,5mg
Viên nén phóng thích kéo dài
55
Betmiga 25mg
Mirabegron
25 mg
Viên nén phóng thích kéo dài
56
Betmiga 50mg
Mirabegron
50mg
Viên nén phóng thích kéo dài
57 Bfluid Injection - Ngăn trên: Mỗi 150ml: LLeucine 2,100g; LIsoleucine 1,200g; LValine 2 100g; L-Lysine
- Ngăn trên: Mỗi
150ml: L-Leucine
2,100g; L-Isoleucine
1 200g; L-Valine
Dung dịch tiêm truyền tĩnh mạch
(*)
Hydrochloride 1,965g
(tương đương L- Lysine
1,573g); L- Threonine
0,855g; L-Tryptophan
0,300g; L-Methionine
0,585g; Acetylcysteine
0,202g (tương đương LCysteine 0,150g); LPhenylalanine 1,050g; LTyrosine 0,075g; LArginine 1,575g; LHistidine 0,750g; LAlanine 1,200g; L- Proline
0,750g; L-Serine 0,450g;
Glycine 0,885g; L-Aspartic acid 0,150g; L- Glutamic acid 0,150g; Dibasic
Potassium Phosphate
0,501g; Dibasic Sodium
Phosphate Hydrate 0,771g;
Dung dịch Sodium LLactate 1,590g (Tương đương Sodium L-Lactate
1,145g); Sodium Citrate
Hydrate 0,285g.
- Ngăn dưới: Mỗi 350ml:
Glucose 37,499g;
Potassium Chloride
0,317g; Calcium Chloride
Hydrate 0,184g;
Magnesium Sulfate
Hydrate 0,308g; Zinc
Sulfate Hydrate 0,70 mg;
Thiamine Chloride
Hydrochloride 0,96 mg
(tương đương Thiamine
0,75 mg).
2,100g; L-Lysine
Hydrochloride
1,965g (tương đương
L- Lysine l,573g); L
Threonine 0,855g;
L-Tryptophan
0,300g; L
Methionine 0,585g;
Acetylcysteine
0,202g (tương đương
L- Cysteine 0,150g);
L-Phenylalanine
1,050g; L- Tyrosine
0,075g; L-Arginine
1,575g; L-Histidine
0,750g; L-Alanine l,200g; L-Proline
0,750g; L-Serine
0,450g; Glycine
0,885g; L-Aspartic acid 0,150g; L
Glutamic acid
0,150g; Dibasic
Potassium Phosphate
0,501g; Dibasic
Sodium Phosphate
Hydrate 0,771g;
Dung dịch Sodium
L-Lactate 1,590g
(Tương đương
Sodium L-Lactate
1,145g); Sodium
Citrate Hydrate
0,285g.
- Ngăn dưới: Mỗi
350ml: Glucose
37,499g; Potassium
Chloride 0,317g;
Calcium Chloride
Hydrate 0,184g;
Magnesium Sulfate
Hydrate 0, 308g;
Zinc Sulfate Hydrate
0,70 mg; Thiamine
Chloride
Hydrochloride 0,96 mg (tương đương
Thiamine 0,75 mg).
Viên nén bao
58 Bifril Zofenopril calci 15mg phim
59
Bifril
Zofenopril calci
30mg
Viên nén bao phim
60
Bilaxten
Bilastine
20mg
Viên nén
61
Bondronat
Sodium Ibandronate monohydrate
6mg Ibandronic acid/6ml
Dung dịch đậm đặc để tiêm truyền
()
62
Bonviva
Mỗi ống tiêm đóng sẵn 3 ml dung dịch chứa Ibandronic acid (dưới dạng Ibandronic acid monosodium salt monohydrate)
Mỗi ống tiêm đóng sẵn 3ml dung dịch chứa Ibandronic acid (dưới dạng Ibandronic acid monosođium salt monohydrate) 3mg
Dung dịch tiêm
63
Brexin
Piroxicam (dưới dạng Piroxicam beta cyclodextrin)
20mg
Viên nén
()
64
Bricanyl
Terbutaline sulphate
0,5mg/ml
Dung dịch tiêm
()
65
Bridion
Sugammadex (dưới dạng sugammadex natri)
200mg/2ml
Dung dịch tiêm tĩnh mạch
66
Brilinta
Ticagrelor
90mg
Viên nén bao phim
()
67
Brilinta
Ticagrelor
60mg
Viên nén bao phim
68
Broncho-Vaxom Adults
Chất ly giải vi khuẩn đông khô tiêu chuẩn 40 mg tương đương chất ly giải vi khuẩn đông khô của Haemophilus influenzae, Streptococcus (Diplococcus) pneumoniae, Klebsiella pneumoniae ssp. pneumonia và ssp. ozaenae, Staphylococcus aureus, Streptococcus pyogenes và sanguinis (viridans), Moraxella (Branhamella/Neisseria) catarrhalis 7 mg
7mg
Viên nang cứng
(*)
69 Broncho-Vaxom
Children
Chất ly giải vi khuẩn đông khô tiêu chuẩn 20 mg tương đương chất ly giải vi khuẩn đông khô của
Haemophilus influenzae, Streptococcus
(Diplococcus) i Kl b i ll
3,5mg Viên nang cứng (*) pneumoniae ssp. pneumonia và ssp. ozaenae, Staphylococcus aureus, Streptococcus pyogenes và sanguinis
(viridans), Moraxella
(Branliamella/Neisseria) catarrhalis 3,5 mg
aureus, Streptococcus pyogenes và sanguinis (viridans), Moraxella (Branliamella/Neisseria) catarrhalis 3,5 mg
70
Bronuck Ophthalmie solution 0.1 %
Bromfenac natri hydrat
1mg/1ml
Dung dịch nhỏ mắt
71
Buscopan
Hyoscin butylbromid
10mg
Viên nén bao đường
72
Calquence
Acalabrutinib
100mg
Viên nang cứng
73
Campto
Irinotecan hydrochlorid trihydrate
100mg/5ml
Dung dịch đậm đặc để pha dung dịch tiêm truyền
74
Campto
Irinotecan hydrochlorid trihydrate
40mg/2ml
Dung dịch đậm đặc để pha dung dịch tiêm truyền
75
Cancidas
Caspofungin (dưới dạng caspofungin acetate 77,7 mg)
70mg
Bột pha dung dịch tiêm truyền
76
Cancidas
Caspofungin (dưới dạng caspofungin acetate 55,5 mg)
50mg
Bột pha dung dịch tiêm truyền
77
Canesten
Clotrimazole
100mg
Viên đặt âm đạo
()
78
Carduran
Doxazosin
2mg
Viên nén
79
Casodex
Bicalutamide
50mg
Viên nén bao phim
80
Cataflam 25
Diclofenac kali
25mg
Viên nén bao đường
()
81
Cataflam 50
Diclofenac kali
50mg
Viên nén bao đường
82
Cavinton
Vinpocetine
10mg/2ml
Dung dịch tiêm
83
Cavinton
Vinpocetine
5mg
Viên nén
84
Cavinton forte
Vinpocetine
10mg
Viên nén
85
Cebrex
Cao khô lá Ginkgo Biloba
40mg
Viên nén bao phim
86
Cebrex S
Cao khô lá Ginkgo Biloba
80mg
Viên nén bao phim
()
87
Ceclor
Cefaclor (dưới dạng Cefaclor monohydrate)
250mg
Viên nang cứng
()
88
Ceclor
Cefaclor (dưới dạng Cefaclor monohydrate)
375mg
Viên nén bao phim giải phóng chậm
89
Ceclor
Cefaclor (dưới dạng Cefaclor monohydrate)
125mg/5ml
Cốm pha hỗn dịch uống
90
Cefobid
Cefoperazone (dưới dạng Cefoperazone Natri)
1g
Bột pha dung dịch tiêm
91
Celebrex
Celecoxib
200mg
Viên nang cứng
92
Cellcept
Mycophenolate mofetil
500mg
Viên nén bao phim
93
Cellcept
Mycophenolate mofetil
250mg
Viên nang cứng
94
Cerebrolysin
Peptides (Cerebrolysin concentrate)
215,2mg/ml
Dung dịch tiêm và truyền
95
Certican 0.25mg
Everolimus
0,25mg
Viên nén
96
Certican 0.5mg
Everolimus
0,5mg
Viên nén
97
Certican 0.75mg
Everolimus
0,75mg
Viên nén
98
Cialis
Tadalafil
5mg
Viên nén bao phim
99
Cialis
Tadalafil
20mg
Viên nén bao phim
100
Ciprobay 200
Ciprofloxacin
200mg/100ml
Dung dịch truyền tĩnh mạch
101
Ciprobay 400mg
Ciprofloxacin
400mg/200ml
Dung dịch truyền tĩnh mạch
102
Ciprobay 500
Ciprofloxacin (dưới dạng Ciprofloxacin hydrochlorid)
500mg
Viên nén bao phim
103
Clamoxyl 250mg
Amoxicillin (dưới dạng Amoxicillin trihydrat)
250mg
Bột pha hỗn dịch uống
104
Coaprovel
Irbesartan; Hydrochlorothiazide
150mg; 12,5mg
Viên nén bao phim
105
CoAprovel® 300/12.5mg
Irbesartan; Hydrochlorothiazide
300mg; 12,5mg
Viên nén bao phim
106
Co-Diovan 160/25
Valsartan; Hydrochlorothiazide
160mg; 25mg
Viên nén bao phim
107
Co-Diovan 80/12,5
Valsartan; Hydrochlorothiazide
80mg; 12,5mg
Viên nén bao phim
108
Coltramyl 4mg
Thiocolchicoside
4mg
Viên nén
109 Combigan Brimonidine tartrate; (2mg; 5mg)/ml Dung dịch nhỏ
Timolol (dưới dạng Timolol maleate)
mắt
110
Combivent
Ipratropium bromide anhydrous (dưới dạng ipratropium bromide monohydrate); Salbutamol (dưới dạng salbutamol sulfate)
0,5mg; 2,5mg
Dung dịch khí dung
111
Concerta
Methylphenidat hydrochlorid
18mg
Viên nén phóng thích kéo dài
112
Concerta
Methylphenidat hydrochlorid
27mg
Viên nén phóng thích kéo dài
113
Concerta
Methylphenidat hydrochlorid
36mg
Viên nén phóng thích kéo dài
114
Concor 5mg
Bisoprolol fumarate
5 mg
Viên nén bao phim
115
Concor Cor
Bisoprolol fumarate
2,5mg
Viên nén bao phim
116
Cordarone
Amiodarone hydrochloride
200mg
Viên nén
117
Cordarone 150mg/3ml
Amiodarone hydrochloride
150mg/3ml
Dung dịch tiêm
118
Coversyl 10mg
Perindopril arginine (tương ứng với 6,790mg perindopril)10mg
10mg
Viên nén bao phim
119
Coversyl 5mg
Perindopril arginine (tương ứng với 3,395mg perindopril) 5mg
5mg
Viên nén bao phim
120
Cozaar 100mg
Losartan potassium
100mg
Viên nén bao phim
121
Cozaar 50mg
Losartan potassium
50mg
Viên nén bao phim
122
Cravit
Levofloxacin hydrat
25mg/5ml
Dung dịch nhỏ mắt
123
Cravit 1.5%
Levofloxacin hydrat
75mg/5ml
Dung dịch nhỏ mắt
124
Crestor
Rosuvastatin (dưới dạng Rosuvastatin calci)
5mg
Viên nén bao phim
125
Crestor 10mg
Rosuvastatin (dưới dạng Rosuvastatin calci)
10mg
Viên nén bao phim
126
Crestor 20mg
Rosuvastatin (dưới dạng Rosuvastatin calci)
20mg
Viên nén bao phim
Phospholipid chiết xuất từ
127 Curosurf 120mg/1,5ml phổi lợn
Hỗn dịch bơm ống nội khí
128
Cưrosurf
Phospholipid chiết xuất từ phổi lợn
240mg/3ml
Hỗn dịch bơm ống nội khí quản
()
129
Daflon 500mg
Phân đoạn flavonoid vi hạt tinh chế ứng với: Diosmin 90%; Các flavonoid biểu thị bằng hesperidin 10%
450mg; 50mg
Viên nén bao phim
130
Daivonex
Calcipotriol
50mcg/g
Thuốc mỡ
131
Dalacin C
Clindamycin (dưới dạng Clindamycin Hydrochloride)
300mg
Viên nang cứng
132
Dalacin C
Clindamycin (dưới dạng Clindamycin phosphat)
300mg/2ml
Dung dịch tiêm/truyền
133
Dalacin C
Clindamycin (dưới dạng Clindamycin phosphate)
600mg/4ml
Dung dịch tiêm/truyền
134
Daxas
Roflumilast micronized
0,5mg
Viên nén bao phim
135
Debridat
Trimebutine maleate
100mg
Viên nén bao phim
136
Depakine Chrono
Natri valproate; Acid valproic
333mg; 145mg
Viên nén bao phim phóng thích kéo dài
137
Depo-Medrol
Methylprednisolon acetat
40mg/ml
Hỗn dịch tiêm
138
Dermovate cream
Clobetasol propionat
0,05% khối lượng/khối lượng
Kem bôi ngoài da
139
Diamicron MR
Gliclazide
30mg
Viên nén phóng thích có kiểm soát
140
Diamicron MR 60mg
Gliclazide
60mg
Viên nén phóng thích có kiếm soát
141
Diflucan
Fluconazole
150mg
Viên nang cứng
142
Diflucan IV
Fluconazole
200mg/100ml
Dung dịch truyền tĩnh mạch
143
Dilatrend
Carvedilol
6,25 mg
Viên nén
()
144
Dilatrend
Carvedilol
12,5mg
Viên nén
()
145
Dilatrend
Carvedilol
25 mg
Viên nén
()
146
Diovan 160
Valsartan
160mg
Viên nén bao phim
Viên nén bao
147 Diovan 80 Valsartan 80mg phim
148
Diprivan
Propofol
10mg/ml (1% w/v) (Bơm tiêm 50ml)
Nhũ tương tiêm/truyền tĩnh mạch
()
149
Diprivan
Propofol
10mg/ml (Ống 20ml)
Nhũ tương tiêm/truyền tĩnh mạch
()
150
Diquas
Natri diquafosol
30mg/ml
Dung dịch nhỏ mắt
151
Duodart
Dutasteride; Tamsulosin hydrochloride
0,5mg; 0,4mg
Viên nang cứng
152
Duoplavin
Clopidogrel (dưới dạng Clopidogrel hydrogen sulphat); Acid acetyIsalicylic
75mg; 100mg
Viên nén bao phim
153
Duphaston
Dydrogesterone
10mg
Viên nén bao phim
154
Durogesic 25mcg/h
Fentanyl
4,2mg
Miếng dán phóng thích qua da
155
Durogesic 50mcg/h
Fentanyl
8,4mg
Miếng dán phóng thích qua da
156
Duspatalin retard
Mebeverin hydroclorid
200mg
Viên nang cứng giải phóng kéo dài
157
Efient Film- coated tablet
Prasugrel
10mg
Viên nén bao phim
(*)
158
Eliquis
Apixaban
5mg
Viên nén bao phim
159
Eliquis
Apixaban
2,5mg
Viên nén bao phim
160
Eloxatin
Oxaliplatin
100mg/20ml
Dung dịch đậm đặc pha dung dịch tiêm truyền tĩnh mạch
161
Eloxalin
Oxaliplatin
50mg/10ml
Dung dịch đậm đặc pha dung dịch tiêm truyền tĩnh mạch
162
Elthon 50mg
Itoprid hydrochloride
50mg
Viên nén bao phim
163
Emla
Lidocain; Prilocain
125mg/5g; 125mg/5g
Kem bôi
 f
165
Eprex 2000 U
Epoetin alfa
2000 IU/0,5ml
Dung dịch tiêm
166
Eprex 4000 U
Epoetin alfa
4000 IU/0,4ml
Dung dịch tiêm
167
Eraxis
Anidulafujgin
100mg
Bột đông khô pha dung dịch truyền
168
Erbitux
Cetuximab
5mg/ml
Dung dịch tiêm truyền
169
Esmeron
Rocuronium bromide
10mg/ml
Dung dịch tiêm
170
Eumovate cream
Clobetasone butyrate (dưới dạng micronised)
0,05% (kl/kl)
Kem bôi ngoài da
()
171
Exforge
Amlodipine (dưới dạng Amlodipine besylate); Valsartan
10mg; 160mg
Viên nén bao phim
172
Exforge
Amlodipine (dưới dạng Amlodipine besylate); Valsartan
5mg; 80mg
Viên nén bao phim
173
Exforge HCT 10mg/160mg/12.5mg
Amlodipin (dưới dạng amlodipin besilate); Valsartan; Hydrochlorothiazide
10mg; 160mg; 12,5mg
Viên nén bao phim
174
Exforge HCT 10mg/320mg/25mg
Amlodipin (dưới dạng amlodipin besilate); Valsartan; Hydrochlorothiazide
10mg; 320mg; 25mg
Viên nén bao phim
()
175
Exforge HCT 5mg/160mg/12.5 mg
Amlodipin (dưới dạng amlodipin besilate); Valsartan; Hydrochlorothiazide
5mg; 160mg; 12,5mg
Viên nén bao phim
176
Exjade 125
Deferasirox
125mg
Viên nén phân tán
177
Exjade 250
Deferasirox
250mg
Viên nén phân tán
178
Eylea
Aflibercept
40mg/ml
Dung dịch tiêm nội nhãn
179
Faslodex
Fulvestrant
50mg/ml
Dung dịch tiêm
180
Femara
Letrozole
2,5mg
Viên nén bao phim
181
Flixonase
Fluticason propionat (siêu mịn)
0,05% (w/w)
Hỗn dịch xịt mũi
182 Flixotide Evohaler Fluticasone propionate 125mcg/liều xịt
Thuôc xịt dạng phun mù định liều (dùng để hít qua miệng)
183
Flumetholon 0.02
Fluorometholon
1mg/5ml
Hỗn dịch nhỏ mắt
184
Flumetholon 0.1
Fluorometholon
5mg/5ml
Hỗn dịch nhỏ mắt
185
Fortum
Ceftazidim (dưới dạng Ceftazidim pentahydrat)
1g
Bột pha tiêm hay truyền
186
Forxiga
Dapagliflozin (dưới dạng Dapagliflozin propanediol monohydrat)
5mg
Viên nén bao phim
187
Forxiga
Dapagliflozin (dưới dạng Dapagliflozin propanediol monohydrat)
10mg
Viên nén bao phim
188
Fosmicin for l.V.Use 1g
Fosfomycin sodium
1 g (hoạt lực)
Bột pha tiêm
189
Fosmicin for l.V.Use 2g
Fosfomycin sodium
2g (hoạt lực)
Bột pha tiêm
190
Fosmicin tablets 500
Fosfomycin Calcium hydrate
500mg (hoạt lực)
Viên nén
191
Fosmicin-S for Otic
Fosfomycin sodium
300mg (hoạt lực)
Bột pha dung dịch thuốc nhỏ tai
192
Fucidin
Acid Fusidic
2%
Kem
193
Fycompa 2 mg
Perampanel
2mg
Viên nén bao phim
194
Fycompa 4 mg
Perampanel
4mg
Viên nén bao phim
195
Fycompa 8 mg
Perampanel
8mg
Viên nén bao phim
196
Gadovist
Gadobutrol
1mmol/ml (604,72mg/ml) x 7,5 ml
Dung dịch tiêm
197
Gadovist
Gadobutrol
604,72mg tương đương 1 mmol/ml x 5ml
Dung dịch tiêm
198
Galvus
Vildagliptin
50mg
Viên nén
199
Galvus Met 50mg/1000mg
Vildagliptin; Metformin hydrochloride
50mg; 1000mg
Viên nén bao phim
200
Galvus Met 50mg/500mg
Vildagliptin; Metformin hydrochloride
50mg: 500mg
Viên nén bao phim
201
Galvus Met 50mg/850tng
Vildagliptin; Metformin hydrochloride
50mg; 850mg
Viên nén bao phim
202 Ganfort
Bimatoprost; Timolol
(dưới dạng Timolol maleate)
Dung dịch nhỏ
0,3mg/ml; 5mg/ml mắt
203
Gemzar
Gemcitabin (dưới dạng Gemcitabine hydrochloride)
1000mg
Bột đông khô để pha dung dịch tiêm truyền
204
Gemzar
Gemcitabin (dưới dạng Gemcitabine hydrochloride)
200mg
Bột đông khô để pha dung dịch tiêm truyền
205
Glivec 100mg
Imatinib (dưới dạng lmatinib mesylat 119,5mg)
100mg
Viên nén bao phim
206
Glucobay 100mg
Acarbose
100mg
Viên nén
207
Glucobay 50mg
Acarbose
50mg
Viên nén
208
Glucophage
Metformin hydrochlorid
1000mg
Viên nén bao phim
209
Glucophage 500mg
Metformin hydrochloriđ
500mg
Viên nén bao phim
210
Glucophage 850mg
Metformin hydrochlorid
850mg
Viên nén bao phim
211
Glucophage XR 1000mg
Metformin hydrochlorid
1000mg
Viên nén phóng thích kéo dài
212
Glucophage XR 750mg
Metformin hydrochlorid
750mg
Viên nén phóng thích kéo dài
213
Glucovance 500m g/2,5mg
Metformin hydrochlorid; Glibenclamid
500mg; 2,5mg
Viên nén bao phim
214
Glucovance 500mg/5mg
Metformin hydrochlorid; Glibenclamid
500mg; 5mg
Viên nén bao phim
215
Grandaxin
Tofisopam
50mg
Viên nén
216
Giotrif
Afatinib (dưới dạng Afatinib dimaleate)
40mg
Viên nén bao phim
217
Giotrif
Afatinib (dưới dạng Afatinib dimaleate)
30mg
Viên nén bao phim
218
Giotrif
Afatinib (dưới dạng Afatinib dimaleate)
20mg
Viên nén bao phim
219
Giotrif
Afatinib (dưới dạng Afatinib dimaleate)
50mg
Viên nén bao phim
220
Harnal Ocas 0,4mg
Tamsulosin hydroclorid
0,4mg
Viên nén bao phim phóng thích chậm
221
Herceptin
Trastuzumab
150mg
Bột đông khô để pha dung dịch truyền
222 Herceptin Trastuzumab 440mg
Bột đông khô để pha dung dịch tiêm truyền
223
Herceptin
Trastuzumab
600mg
Dung dịch tiêm
224
Hidrasec 100mg
Racecadotril
100mg
Viên nang cứng
()
225
Hidrasec 10mg Infants
Racecadotril
10mg
Thuốc bột uống
()
226
Hidrasec 30mg Children
Racecadotril
30mg
Thuốc bột uống
()
227
Humalog Kwikpen
Insulin lispro
300U/3ml (tương đương 10,5mg)
Dung dịch tiêm
228
Humalog Mix 50/50 Kwikpen
Insulin lispro (trong đó 50 % là dung dịch insulin lispro và 50% là hỗn dịch insulin lispro protamine)
300U/3ml
Hỗn dịch tiêm
229
Humalog Mix 75/25 Kwikpen
Insulin lispro (trong đó 75% là dung dịch insulin lispro và 25% là hỗn dịch insulin lispro protamine)
300U/3ml
Hỗn dịch tiêm
230
Humira
Adalimumab
40mg/0,8ml
Dung dịch tiêm
()
231
Hyalgan
Natri hyaluronate
20mg/2ml
Dung dịch tiêm trong khớp
232
Hyperium
Rilmenidine dihydrogen phosphate 1,544mg tương đương Rilmenidine 1mg
1mg
Viên nén
233
Hyzaar 50mg/12.5mg
Losartan potassium; Hydrochlorothiazide
50mg; 12,5mg
Viên nén bao phim
234
Ibrance 100mg
Palbociclib
100mg
Viên nang cứng
()
235
Ibrance 125mg
Palbociclib
125mg
Viên nang cứng
236
Ibrance 75mg
Palbociclib
75mg
Viên nang cứng
237
Ilomedin 20
Iloprost (dưới dạng Iloprost trometamol)
20mcg/ml
Dung dịch đậm đặc pha tiêm truyền
238
Imbruvica
Ibrutinib
140mg
Viên nang cứng
239
Imodium
Loperamide hydrocloride
2mg
Viên nang
()
240
Inlyta 5mg
Axitinib
5mg
Viên nén bao phim
241
Invanz
Ertapenem (dưới dạng Ertapenem natri)
1g
Bột pha dung dịch tiêm truyền
242
Invega Sustenna
Ống tiêm 1,5ml chứa: Paliperidone palmitate 234mg (Tương đương với 150mg paliperidone)
150mg/1,5ml
Hỗn dịch tiêm giải phóng kéo dài
243 Invega Sustenna Ống tiêm 1ml chứa:
Paliperidone palmitate
100mg/1ml Hỗn dịch tiêm giải phóng kéo
100mg paliperidone)
244
Invega Sustenna
Ống tiêm 0,75ml chứa: Paliperidone palmitate (Tương đương với 75 mg paliperidone) 117mg
75mg/0,75ml
Hỗn dịch tiêm giải phóng kéo dài
245
Invega Sustenna
Ống tiêm 0,5ml chứa: Paliperidone palmitate (Tương đương với 50mg paliperidone) 78mg
50mg/0,5ml
Hỗn dịch tiêm giải phóng kéo dài
246
lopamiro
lod (dưới dạng lopamidol 612,4mg/ml)
300mg/ml
Dung dịch tiêm
247
lopamiro
Iod (dưới dạng lopamidol 755,3mg/ml)
370mg/ml
Dung dịch tiêm
248
Iressa
Gefitinib
250mg
Viên nén bao phim
249
Jadenu 180mg
Deferasirox
180mg
Viên nén bao phim
250
Jadenu 360mg
Deferasirox
360mg
Viên nén bao phim
251
Jadenu 90mg
Deferasirox
90mg
Viên nén bao phim
252
Jakavi 10mg
Ruxolitinib (dưới dạng ruxolitinib phosphate)
10mg
Viên nén
253
Jakavi 15mg
Ruxolitinib (dưới dạng ruxolitinib phosphate)
15mg
Viên nén
254
Jakavi 20mg
Ruxolitinib (dưới dạng ruxolitinib phosphate)
20mg
Viên nén
255
Jakavi 5mg
Ruxolitinib (dưới dạng ruxolitinib phosphate)
5mg
Viên nén
256
Janumet 50mg/1000mg
Sitagliptin (dưới dạng Sitagliptin phosphate monohydrate); Metformin hydrochloride
50mg; 1000mg
Viên nén bao phim
257
Janumet 50mg/500mg
Sitagliptin (dưới dạng Sitagliptin phosphate monohydrate); Metformin hydrochloride
50mg; 500mg
Viên nén bao phim
258
Janumet 50mg/850mg
Sitagliptin (dưới dạng Sitagliptin phosphate monohydrate); Metformin hydrochloride
50mg; 850mg
Viên nén bao phim
259 Janumet XR
100mg/1000mg
Sitagliptin (dưới dạng
Sitagliptin phosphate f
100mg; 1000mg Viên nén bao phim giải phóng
HCl
260
Janumet XR 50mg/1000mg
Sitagliptin (dưới dạng Sitagliptin phosphate monohydrate); Metformin HCl
50mg; 1000mg
Viên nén bao phim giải phóng chậm
261
Janumet XR 50mg/500mg
Sitagliptin (dưới dạng Sitagliptin phosphate monohydrate); Metformin HCl
50mg; 500mg
Viên nén bao phim giải phóng chậm
262
Januvia 100mg
Sitagliptin (dưới dạng sitagliptin phosphate monohydrate)
100mg
Viên nén bao phim
263
Januvia 50mg
Sitagliptin (dưới dạng sitagliptin phosphate monohydrate)
50mg
Viên nén bao phim
264
Jardiance
Empagliflozin
25mg
Viên nén bao phim
265
Jardiance
Empagliflozin
10mg
Viên nén bao phim
266
Kadcyla
Trastuzumab emtansine
100mg
Bột pha dung dịch đậm đặc để pha dung dịch tiêm truyền/Bột pha dung dịch tiêm
()
267
Kadcyla
Trastuzumab emtansine
160mg
Bột pha dung dịch đậm đặc để pha dung dịch tiêm truyền/Bột pha dung dịch tiêm
()
268
Kary Uni
Pirenoxin
0,25mg/5ml
Hỗn dịch nhỏ mắt
269
Keppra
Levetiracetam
500mg
Viên nén bao phim
270
Keppra 250mg
Levetiracetam
250mg
Viên nén bao phim
271 Ketosteril Acid (RS)-3-methyl-2oxovaleric (αketoanalogue to DLisoleucin), muối calci;
Acid 4-methyl-2oxovalcric (αketoanalogue to Ieucin), muối calci; Acid 2-oxo-3 h l (
Acid (RS)-3-methyl
2-oxovaleric (α ketoanalogue to DL isoleucin), muối calci 67,0 mg; Acid
4-methyl-2 oxovaleric (α ketoanalogue to l ) ố l
Viên nén bao phim ketoanalogue to
Phenylalanin), muối calci;
Acid 3-methyl-2oxobutvric (αketoanalogue to valin), muối calci; Acid (RS)-2hydroxy-4-methylthiobutyric (αhydroxyanalogue to DLmethionin), muối calci; Llysin acetat (tương đương với L-lysin); L-threonin;
L-tryptophan; L-histidin;
L-tyrosin
101,0 mg; Acid 2 oxo-3 phenylpropionic (α ketoanalogue to phenylalanin), muối calci 68,0 mg; Acid
3-methyl-2 oxobulyric (α ketoanalogue to valin). muối calci
86,0 mg; Acid (RS)
2- hydroxy-4 methylthio-butyric
(α- hydroxyanalogue to DL-melhionin), muối calci 59,0 mg;
L-lysin acetat (tương đương với 75 mg L
L-tyrosin
lysin) 105,00 mg; L- threonin 53,0 mg; L- tryptophan 23,0 mg; L-histidin 38,0 mg; L-tyrosin 30,0 mg
272
Klacid 250mg
Clarithromycin
250mg
Viên nén bao phim
273
Klacid 500mg
Clarithromycin
500mg
Bột pha dung dịch tiêm truyền
274
Klacid Forte
Clarithromycin
500mg
Viên nén bao phim
275
Klacid MR
Clarithromycin
500mg
Viên nén giải phóng biến đổi
276
Komboelyze XR
Saxagliptin; Metformin hydroclorid
5mg; 1000mg
Viên nén bao phim
277
Komboglyze XR
Saxagliptin; Metformin hydroclorid
5mg; 500mg
Viên nén bao phim
278
Komboglyze XR
Saxagliptin; Metformin hydroclond
2,5mg; 1000mg
Viên nén bao phim
279
Kryxana
Ribociclib (dưới dạng Ribociclib succinate)
200mg
Viên nén bao phim
()
280
Lacipil 2mg
Lacidipine
2mg
Viên nén bao phim
281
Lacipil 4mg
Lacidipine
4mg
Viên nén bao phim
282
Lamictal 100mg
Lamotrigine
100mg
Viên nén
()
283
Lamictal 25mg
Lamotrigine
25mg
Viên nén
(*)
284
Lamictal 50mg
Lamotrigine
50mg
Viên nén
(*)
285
Lamisil
Terbinafme hydrochloride
10mg/lg
Kem bôi ngoài da
286
Lantus
Insulin Glargine
100 đơn vị/ml (1000 đơn vị/lọ 10 ml)
Dung dịch tiêm
287
Lantus Solostar
Insulin Glargine
100 đơn vị/1ml
Dung dịch tiêm trong bút tiêm nạp sẵn
288
Lenvima 10mg
Lenvatinib (dưới dạng lenvatinib mesilate)
10mg
Viên nang cứng
289
Lenvima 4mg
Lenvatinib (dưới dạng lenvatinib mesilate)
4mg
Viên nang cúng
290
Levemir FlexPen
Insulin detemir (rDNA)
300 U/3 ml
Dung dịch tiêm
291
Lipanthyl 200M
Fenofibrate
200mg
Viên nang cứng
292
Lipanthyl NT 145mg
Fenofibrate
145mg
Viên nén bao phim
293
Lipanthyl supra 160mg
Fenofibrate
160mg
Viên nén bao phim
294
Lipidem
Medium-chain triglycerides; Soya-bean oil, refined; Omega-3-acid triglycerides
Mỗi 100ml chứa: Medium-chain triglycerides 10,0g; Soya-bean oil, refined 8,0g; Omega-3-acid triglycerides 2,0g
Nhũ tương tiêm truyền
295
Lipitor
Atorvastatin
20mg
Viên nén bao phim
296
Lipitor
Atorvastatin
10mg
Viên nén bao phim
297
Lipitor
Atorvastatin
40mg
Viên nén bao phim
298
Lipofundin MCT/LCT 10%
Medium-chain Triglycerides; Soya- bean Oil
Mỗi 100ml chứa: Medium-chain Triglycerides 5,0g; Soya-bean oil 5,0 g
Nhũ tương tiêm truyền tĩnh mạch
299
Lipofundin MCT/LCT 20%
Medium-chain Triglycerides; Soya- bean Oil
Mỗi 100ml chứa: Medium-chain Triglycerides 10,0g; Soya-bean oil 10,0g
Nhũ tương tiêm truyền tĩnh mạch
300
Lixiana
Edoxaban (dưới dạng Edoxaban tosilat)
15mg
Viên nén bao phim
Edoxaban (dưới dạng Viên nén bao
301 Lixiana 30mg
Edoxaban tosilat) phim
302
Lixiana
Edoxaban (dưới dạng Edoxaban tosilat)
60mg
Viên nén bao phim
303
Losec Mups
Omeprazol (dưới dạng Omeprazol magnesi)
20mg
Viên nén kháng dịch dạ dày
304
Lotemax
Loteprednol etabonate
0,5% (5mg/ml)
Hỗn dịch nhỏ mắt
305
Lovenox
Enoxaparin natri
4000 anti-Xa IU/0,4ml (40mg/0,4ml)
Dung dịch tiêm
306
Lovenox
Enoxaparin natri
6000 anti-Xa IU/0,6ml (60mg/0,6ml)
Dung dịch tiêm
307
Lucentis
Ranibizumab
1,65mg/0,165 ml
Dưng dịch tiêm
308
Lucentis
Ranibizumab
2,3mg/0,23ml
Dung dịch tiêm
(*)
309
Lucrin PDS Depot 11.25mg
Leuprorelin acetate
11,25mg
Bột pha hỗn dịch tiêm
310
Lucrin PDS Depot 3.75mg
Leuprorelin acetate
3,75mg
Bột pha hỗn dịch tiêm
311
Lumigan
Bimatoprost
0,3mg/3ml
Dung dịch nhỏ mắt
312
Lusefi 2.5 mg film- coated tablet
Luseogliflozm (dưới dạng luseogiiflozin hydrate)
2,5mg
Viên nén bao phim
313
Lusefi 5 mg film- coated tablet
Luseogliflozin (dưới dạng luseogliflozin hydrate)
5mg
Viên nén bao phim
314
Luvox 100mg
Fluvoxamine maleate
100mg
Viên nén bao phim
315
Lynparza
Olaparib
100mg
Viên nén bao phim
316
Lynparza
Olaparib
150mg
Viên nén bao phim
317
Lyrica
Pregabalin
75mg
Viên nang cứng
318
Lyrica
Pregabalin
150mg
Viên nang cứng
319
Mabthera
Rituximab
100mg/10ml (Lọ 10ml)
Dung dịch đậm đặc để pha tiêm truyền/dung dịch tiêm truyền
320
Mabthera
Rituximab
500mg/50ml (Lọ 50ml)
Dung dịch đậm đặc để pha dung dịch tiêm truyền
321
Mabthera
Rituximab
1400mg/11,7ml
Dung dịch tiêm dưới da
323
Medrol
Methylprednisolone
16mg
Viên nén
324
Meiact 200mg
Cefditoren (dưới dạng Cefditoren Pivoxil)
200mg
Viên nén bao phim
325
Meiact 400mg
Cefditoren (dưới dạng Cefditoren Pivoxil)
400mg
Viên nén bao phim
()
326
Meronem
Meropenem (dưới dạng meropenem trihydrate)
1g
Bột pha dung dịch tiêm/truyền tĩnh mạch
327
Meronem
Meropenem (dưới dạng meropenem trihydrate)
500mg
Bột pha dung dịch tiêm/truyền tĩnh mạch
328
Miacalcic
Calcitonin cá hồi tổng hợp
50 IU/ml
Dung dịch tiêm
329
Micardis
Telmisartan
80mg
Viên nén
330
Micardis
Telmisartan
40mg
Viên nén
331
Minirin
Desmopressin (dưới dạng Desmopressin acetat 0,1 mg)
0,089mg
Viên nén
332
Mircera
Methoxy polyethylene glycol-epoetin beta
30mcg
Dung dịch tiêm
()
333
Mircera
Methoxy polyethylene glycol-epoetin beta
100mcg
Dung dịch tiêm
()
334
Mircera
Methoxy polyethylene glycol-epoetin beta
50mcg
Dung dịch tiêm
()
335
Mobic
Meloxicam
15mg
Viên nén
336
Mobic
Meloxicam
7,5mg
Viên nén
337
Mobic
Meloxicam
15mg/1,5ml
Dung dịch tiêm
(*)
338 Morihepamin L-Isoleucin, L-Leucin, LLysin acetat, L- Methionin, L-Phenylalanin, LThreonin, L-Tryptophan, L-Valin, L-Alanin, LArginin, L-Aspartic acid, L-Histidin, L- Prolin, LSerin, L-Tyrosin, Glycin
Mỗi 100 mL dung dịch chứa: L
Isoleucin 0,920 g, L
Leucin 0,945 g, L
Lysin acetat 0,395 g, L-Methionin 0,044 g, L-Phenylalanin
0,030 g, L- Threonin
0,214 g, L
Tryptophan 0,070 g, L-Valin 0,890 g, L
Alanin 0,840 g, L
Arginin 1,537 g, LAspartic acid 0,020 g, L-Histidin 0,310 g, L-Prolin 0,530 g, L-Serin 0,260 g, L
Tyrosin 0 040 g
Dung dịch truyền tĩnh mạch
Glycin 0,540 g
339
Motilium
Domperidone
1mg/ml
Hỗn dịch uống
340
Mucosta Tablets 100mg
Rebamipid
100mg
Viên nén
()
341
Mycamine for injection 50mg/vial
Micafungin natri
50 mg (dạng hoạt tính)
Bột đông khô pha dung dịch tiêm
342
Myonal 50mg
Eperisone hydrochloride
50mg
Viên nén bao đường
()
343
Nasonex
Mometasone furoate (dưới dạng Mometasone furoate monohydrate)
50mcg/nhát xịt
Hỗn dịch xịt mũi
344
Natrilix SR
Indapamide
1,5mg
Viên nén bao phim giải phóng kéo dài
345
Navelbine
Vinorelbine
10mg/lml
Dung dịch đậm đặc để pha dung dịch tiêm truyền
346
Navelbine 20mg
Vinorelbine
20mg
Viên nang mềm
347
Navelbine 30mg
Vinorelbine
30mg
Viên nang mềm
348
Nebido
Testosteron undecanoate
1000mg/4ml
Dung dịch tiêm
349
Nebilet
Nebivolol (dưới dạng Nebivolol hydrochloride)
5mg
Viên nén
350
Neulastim
Pegfilgrastim
6mg/0,6ml
Dung dịch tiêm
351
Neupogen
Filgrastim
30MU/0,5ml
Dung dịch tiêm
352
Neurontin
Gabapentin
300mg
Viên nang cứng
353
Nevanac
Nepafenac
1mg/ml
Hỗn dịch nhỏ mắt
354
Nexavar
Sorafenib (dạng tosylate)
200mg
Viên nén bao phim
355
Nexium
Esomeprazole (dưới dạng Esomeprazole magnesi trihydrate)
10mg
Cốm kháng dịch dạ dày để pha hỗn dịch uống
356
Nexium
Esomeprazol natri (tương đương với esomeprazol 40mg)
42,5mg
Bột pha dung dịch tiêm/truyền tĩnh mạch
357
Nexium Mups
Esomeprazol (dưới dạng Esomeprazol magnesi trihydrat)
40mg
Viên nén kháng dịch dạ dày
358 Nexium Mups
Esomeprazol (dưới dạng
Viên nén kháng
Esomeprazol magnesi 20mg dịch dạdày
359
Nimotop
Nimodipin
30mg
Viên nén bao phim
360
Nimotop
Nimodipine
10mg/50ml
Dung dịch tiêm truyền
()
361
Nolvadex
Tamoxifen (dưới dạng Tamoxifen citrat)
10mg
Viên nén bao phim
362
Nolvadex-D
Tamoxifen (dưới dạng Tamoxifen Citrat)
20mg
Viên nén bao phim
363
Nootropil
Piracetam
800mg
Viên nén bao phim
364
Norditropin NordiFIex 5mg/1.5ml
Somatropin
3,3mg/ml
Dung dịch tiêm
()
365
No-Spa 40mg/2ml
Drotaverme hydrochloride
40mg/2ml
Dung dịch tiêm
366
No-Spa forte
Drotaverine hydrochloride
80mg
Viên nén
367
NovoMix 30 FlexPen
Insulin aspart biphasic (rDNA) 1ml hỗn dịch chứa 100U của insulin aspart hòa tan/insulin aspart kết tinh với protamine theo tỷ lệ 30/70 (tương đương 3,5mg)
100 U/1ml
Hỗn dịch tiêm
(*)
368
NovoRapid
Insulin aspart
1000 U/10ml
Dung dịch tiêm
369
NovoRapid FlexPen
Insulin aspart (rDNA)
300 U/3ml
Dung dịch tiêm
370
Noxafil
Posaconazole
40mg/ml
Hỗn dịch uống
371
Ofev
Nintedanib (dưới dạng nintedanib esilate)
100mg
Viên nang mềm
372
Ofev
Nintedanib (dưới dạng nintedanib esilate)
150mg
Viên nang mềm
373
Oflovid
Ofloxacin
15mg/5ml
Dung dịch nhỏ mắt
374
Oflovid Ophthalmie Ointment
Ofloxacin
0,3%
Thuốc mỡ tra mắt
375
Omnipaque
Iohexol
755mg/ml (tương đương Iod 350mg/ml)
Dung dịch tiêm
376
Omnipaque
lohexol
647mg/ml (tương đương Iod 300mg/ml)
Dung dịch tiêm
377
Onbrez Breezhaler
Indacaterol (dưới dạng indacaterol maleat)
150mcg
Bột hít chứa trong nang cứng
Indacaterol (dưới dạng Bột hít chứa
378 Onbrez Breezhaler 300mcg indacaterol maíeat) trong nang cứng
379
Onglyza
Saxagliptin
2,5mg
Viên nén bao phim
380
Onglyza
Saxagliptin
5mg
Viên nén bao phim
381
Orgalutran
Ganirelix
0,25mg/0,5ml
Dung dịch tiêm
382
Otrivin
Xylometazoline hydrochloride
5mg/10ml
Dung dịch nhỏ mũi
383
Otrivin
Xylometazoline hydrochloride
10mg/10ml
Dung dịch nhỏ mũi
384
Otrivin
Xylometazoline hydrochloride
15mg/10ml
Thuốc xịt mũi có phân liều
385
Otrivin
Xylometazoline hydrochloride
10mg/10ml
Thuốc xịt mũi có phân liều
386
Pantoloc 20mg
Pantoprazole sodium sesquihydrate (tương đương pantoprazole 20mg)
22,57mg
Viên nén bao tan trong ruột
387
Pantoloc 40mg
Pantoprazole (dưới dạng Pantoprazole sodium sesquihydrate)
40mg
Viên nén bao tan trong ruột
388
Pantoloc i.v
Pantoprazole (dưới dạng Pantoprazole sodium sesquihydrate)
40mg
Bột đông khô pha tiêm tĩnh mạch
389
Pariet 10mg
Rabeprazol (dưới dạng Rabeprazol natri)
10mg
Viên nén bao tan trong ruột
()
390
Pariet 20mg
Rabeprazole sodium
20mg
Viên nén bao tan trong ruột
()
391
Pataday
Olopatadine hydrochloride
0,2%
Dung dịch nhỏ mắt
392
Perjeta
Pertuzumab
420mg/14 ml
Dung dịch đậm đặc để pha dung dịch tiêm truyền
393
Plavix
Clopidogrel hydrogen sulphate tương đương với Clopidogrel base
300mg
Viên nén bao phim
394
Plavix 75mg
Clopidogrel hydrogen sulphate tương đương với Clopidogrel base
75mg
Viên nén bao phim
395
Plendil
Felodipin
5 mg
Viên nén phóng thích kéo dài
(*)
396
Pradaxa
Dabigatran etexilate (dưới dạng Dabigatran etexilatc mesilatc)
110 mg
Viên nang cứng
dạng Dabigatran etexilate mesilate)
398
Pradaxa
Dabigatran etexilate (dưới dạng Dabigatran etexilate mesilate)
75mg
Viên nang cứng
399
Priligy 30mg
Dapoxetine (dưới dạng Dapoxetine hydrochloride)
30mg
Viên nén bao phim
400
Priligy 60mg
Dapoxetine (dưới dạng Dapoxetine hydrochloride)
60mg
Viên nén bao phim
401
Primovist
Gadoxetate disodium
0,25 mmol tương đương 181,43mg/ml x 10ml
Dung dịch tiêm
402
Pristiq
Desvenlafaxin (dưới dạng Desvenlafaxin succinate monohydrate)
50mg
Viên nén giải phóng kéo dài
403
Procoralan 5mg
Ivabradin (dưới dạng Ivabradin hydrochloride) 5mg
5mg
Viên nén bao phim
404
Procoralan 7.5mg
Ivabradin (dưới dạng Ivabradin hydrochloride) 7,5 mg
7,5 mg
Viên nén bao phim
405
Prograf 0.5 mg
Tacrolimus
0,5mg
Viên nang cứng
406
Prograf 1mg
Tacrolimus
1mg
Viên nang cứng
407
Prograf 5mg/ml
Tacrolimus
5mg/ml
Dung dịch đậm đặc để pha truyền
408
Protopic
Tacrolimus
0,03%
Thuốc mỡ
()
409
Protopic
Tacrolimus
0,1%
Thuốc mỡ
()
410
Pulmicort Respules
Budesonid
500mcg/2ml
Hỗn dịch khí dung dùng để hít
411
Pulmicort Respules
Budesonid
1mg/2ml
Hỗn dịch khí dung dùng để hít
412
Phosphalugel
Aluminium phosphate 20% gel
12,38g/gói 20g
Hỗn dịch uống
413
Ranexicor
Ranolazine
750mg
Viên nén giải phóng kéo dài
()
414
Ranexicor
Ranolazine
375mg
Viên nén giải phóng kéo dài
()
415
Ranexicor
Ranolazine
500mg
Viên nén giải phóng kéo dài
(*)
416
Rcagila 1.5 mg
Cariprazine (dưới dạng cariprazine hydrochloride)
1,5mg
Viên nang cứng
417
Reagila 3 mg
Cariprazine (dưới dạng cariprazine hydrochloride)
3mg
Viên nang cứng
418
Reagila 4.5 mg
Cariprazine (dưới dạng cariprazine hydrochloride)
4,5mg
Viên nang cứng
419
Reagila 6 mg
Cariprazine (dưới dạng cariprazine hydrochloride)
6mg
Viên nang cứng
420
Relenza
Zanamivir
5mg/liều
Thuốc bột hít
()
421
Remeron 30
Mirtazapine
30mg
Viên nén bao phim
422
Remicade
Infliximab
100mg
Bột pha dung dịch đậm đặc để pha dung dịch truyền
423
Renvela
Sevelamer carbonate
800mg
Viên nén bao phim
()
424
Restasis
Cyclosporine
0,05% (0,5mg/g)
Nhũ tương nhỏ mắt
425
Revolade 25mg
Eltrombopag (dưới dạng Eltrombopag olamine)
25mg
Viên nén bao phim
426
Ribomustin
Bendamustine hydrochloride (tương đương 22,7 mg Bendamustine)
25mg
Bột đông khô pha dung dịch đậm đặc để pha dung dịch truyền tĩnh mạch
427
Ribomustin
Bendamustine hydrochloride (tương đương 90,8 mg Bendamustine)
100mg
Bột đông khô pha dung dịch đậm đặc để pha dung dịch truyền tĩnh mạch
428
Risperdal
Risperidone
2mg
Viên nén bao phim
429
Risperdal
Risperidone
1mg
Viên nén bao phim
430
Rocephin 1g I.V
Ceftriaxone (dưới dạng Ceftriaxone natri)
1g
Thuốc bột pha tiêm
431
Rulid 150mg
Roxithromycin
150mg
Viên nén bao phim
432
Rupafin
Rupatadine (dưới dạng Rupatadine fumarate)
10mg
Viên nén
433
Ryzodeg FlexTouch 100U/ml
aspart; Mỗi bút tiêm bơm sẵn 3ml chứa 7,68mg Insulin degludec và 3,15mg Insulin aspart
(7,68mg; 3,15mg)/3ml
Dung dịch tiêm
434
Ryzodeg Penfill 100U/ml
Insulin degludec; Insulin aspart; Mỗi ống 3ml chứa 7,68mg Insulin degludec và 3,15mg Insulin aspart
(7,68mg; 3,15mg)/3ml
Dung dịch tiêm
435
Salzen liquid
Somatropin
6mg
Dung dịch tiêm
436
Sandimmun
Ciclosporin
50mg/ml
Dung dịch đậm đặc để pha truyền tĩnh mạch
437
Sandimmun Neoral
Ciclosporin
100mg/ml
Dung dịch uống
438
Sandimmun Neoral 100mg
Ciclosporin
100mg
Viên nang mềm
439
Sandimmun Neoral 25mg
Ciclosporin
25mg
Viên nang mềm
440
Sandostatin
Octreotide
0,1mg/ml
Dung dịch tiêm
441
Sandostatin Lar 20mg
Octreotid (dưới dạng Octreotid acetat)
20 mg
Bột pha tiêm
442
Sandostatin Lar 30mg
Octreotid (dưới dạng Octreotid acetat)
30 mg
Bột pha tiêm
443
Sanlein 0.1
Natri hyaluronat tinh khiết
5mg/5ml
Dung dịch nhỏ mắt
444
Sanlein 0.3
Natri hyaluronat tinh khiết
15mg/5ml
Dung dịch nhỏ mắt
445
Sanlein Mini 0.1
Natri hyaluronat tinh khiết
0,4mg/0,4ml
Dung dịch nhỏ mắt
446
Sanlein Mini 0.3
Natri hyaluronat tinh khiết
1,2mg/0,4ml
Dung dịch nhỏ mắt
447
Saxenda 6mg/ml
Liraglutide
18mg/3ml
Dung dịch tiêm
448
Sayana Press
Medroxyprogesterone acetate
104mg/0,65ml
Hỗn dịch tiêm
449
Seretide Accuhaler 50/250mcg
Salmeterol (dưới dạng Salmeterol xinafoate): Fluticason propionat
Mỗi liều hít chứa: Salmeterol (dưới dạng Salmeterol xinafoate) 50mcg; Fluticason propionat 250mcg
Bột hít phân liều
Mỗi liều hít chứa:
Salmeterol (dưới
450 Seretide Accuhaler
50/500mcg
Insulin degludec; Insulin aspart; Mỗi bút tiêm bơm sẵn 3ml chứa 7,68mg
Insulin degludec và
3,15mg Insulin aspart
Insulin degludec; Insulin aspart; Mỗi ống 3ml chứa
7,68mg Insulin degludec và 3,15mg Insulin aspart
Somatropin
Ciclosporin
Ciclosporin
Ciclosporin
Ciclosporin
Octreotide
Octreotid (dưới dạng
Octreotid acetat)
Octreotid (dưới dạng
Octreotid acetat)
Natri hyaluronat tinh khiết
Natri hyaluronat tinh khiết
Natri hyaluronat tinh khiết
Natri hyaluronat tinh khiết
Liraglutide
Medroxyprogesterone acetate
Salmeterol (dưới dạng
Salmeterol xinafoate):
Fluticason propionat
Salmeterol (dưới dạng
Salmeterol xinafoate);
Bột hít phân liều
Fluticasone propionate
dạng Salmeterol xinafoate) 50mcg; Fluticasone propionate 500mcg
451
Seretide Evohaler DC 25/125mcg
Salmeterol (dạng xinafoate micronised); Fluticasone propionate (dạng micronised)
Mỗi liều xịt chứa: 25 mcg Salmeterol (dạng xinafoate micronised) và 125mcg Fluticasone propionate (dạng micronised)
Hỗn dịch hít qua đường miệng (dạng phun sương)
452
Seretide Evohaler DC 25/250mcg
Salmeterol (dưới dạng Salmeterol xinafoate micronised); Fluticason propionate (dạng micronised)
Mỗi liều xịt chứa: Salmeterol (dưới dạng Salmeterol xinafoate micronised) 25 mcg; Fluticason propionate (dạng micronised) 250mcg
Thuốc phun mù định liều hệ hỗn dịch
453
Seretide Evohaler DC 25/50mcg
Salmeterol xinafoate; Fluticasone propionate
Fluticasone propionate 50mcg; Salmeterol 25mcg/liều
Thuốc phun mù hệ hỗn dịch để hít qua đường miệng
454
Seroquel XR
Quetiapin (dưới dạng Quetiapin fumarat)
200mg
Viên nén phóng thích kéo dài
()
455
Seroquel XR
Quetiapin (dưới dạng Quetiapin fumarat)
300mg
Viên nén phóng thích kéo dài
()
456
Seroquel XR
Quetiapin (dưới dạng Quetiapin fumarat)
50mg
Viên nén phóng thích kéo dài
(*)
457
Sevorane
Sevoflurane
250ml
Dược chất lỏng nguyên chất dùng để hít
458
Sifrol
Pramipexole dihydrochloride monohydrate (tương đương 0,18mg pramipexole)
0,25mg
Viên nén
459
Sifrol
Pramipexole dihydrochloride monohydrate (tương đương 0,26mg pramipexole)
0,375mg
Viên nén phóng thích chậm
460
Sifrol
Pramipexol dihydrochloride monohydrate (tương đương 0,52mg pramipexol)
0,75mg
Viên nén phóng thích chậm
461
Simponi
Golimumab
50mg/0,5ml
Dung dịch tiêm
 l l b ộ h ê
463
Singulair
Montelukast (dưới dạng Montelukast natri)
10mg
Viên nén bao phim
464
Singulair
Montelukast (dưới dạng Montelukast natri)
4mg
Cốm uống
()
465
Singulair 4mg
Montelukast (dưới dạng Montelukast natri)
4mg
Viên nén nhai
466
Singulair 5mg
Montelukast (dưới dạng Montelukast natri)
5mg
Viên nén nhai
467
Solu-Medrol
Methylprednisolon (dưới dạng Methylprednisolon natri succinat)
40mg
Bột đông khô pha tiêm
468
Solu-Medrol
Methylprednisolon (dưới dạng Methylprednisolon natri succinat)
500mg
Bột đông khô pha tiêm
469
Solu-Medrol
Methylprednisolone Hemisuccinat
125mg Methylprednisolon
Bột vô khuẩn pha tiêm
470
Spasmomen
Otilonium bromide
40mg
Viên nén bao phim
471
Spexib 150mg
Ceritinib
150mg
Viên nang cứng
472
Spiriva Respimat
Tiotropium
0,0025mg/nhát xịt
Dung dịch để hít
473
Sporal
Itraconazole
100mg
Viên nang
()
474
Stabion
Tianeptine sodium
12,5mg
Viên nén bao đường
(*)
475
Stelara
Ustekinumab
45mg/0,5ml
Dung dịch tiêm
476
Stivarga
Regorafenib
40mg
Viên nén bao phim
477
Survanta
Phospholipids chiết xuất từ phổi bò
25mg/ml
Hỗn dịch dùng đường nội khí quản
478
Symbicort Rapihaler
Budesonid; Formoterol fumarate dihydrate
(160mcg; 4,5mcg)/liều xịt
Hỗn dịch nén dưới áp suất dùng để hít
479
Symbicort Rapihaler
Budesonid; Formoterol fumarate dihydrate
(80mcg; 4,5mcg)/liều xịt
Hỗn dịch nén dưới áp suất dùng để hít
480
Symbicort Turbuhaler
Budesonid; Formoterol fumarate dihydrate
(160mcg; 4,5mcg)/liều
Thuốc bột để hít
481
Symbicort Turbuhaler
Budesonid; Formoterol fumarate dihydrate
(80mcg; 4,5mcg)/liều
Bột dùng để hít
482 Sympal Dexketoprofen (dưới dạng
Dexketoprofen
50mg/2ml Dung dịch tiêm bắp chậm tiêm
trometamol)
tĩnh mạch chậm, truyền tĩnh mạch sau khi pha
483
Sympal
Dexketoprofen (dưới dạng Dexketoprofen trometamol)
25mg
Viên nén bao phim
484
Taflotan
Tafluprost
0,0375mg/2,5ml
Dung dịch nhỏ mắt
485
Taflotan-S
Tafluprost
4,5mcg/0,3ml
Dung dịch nhỏ mắt
486
Tagrisso
Osimertinib (tương ứng 47,7 mg Osimertinib mesylat)
40mg
Viên nén bao phim
487
Tagrisso
Osimertinib (tương đương 95,4 mg Osimertinib mesylat)
80mg
Viên nén bao phim
488
Tamiflu
Oseltamivir
75 mg
Viên nang cứng
()
489
Tanakan
Dịch chiết ginkgo biloba (Ginkgo biloba extract)
40mg
Viên nén bao phim
490
Tanganil 500mg/5ml
Acetylleucine
500mg/5ml
Dung dịch tiêm tĩnh mạch
()
491
Tarceva
Erlotinib (dưới dạng Erlotinib hydrochloride)
150mg
Viên nén bao phim
492
Tarceva
Erlotinib (dưới dạng Erlotinib hydrochloride)
100mg
Viên nén bao phim
493
Tasigna 150mg
Nilotinib (dưới dạng Nilotinib hydrochloride monohydrate)
150mg
Viên nang cứng
494
Tasigna 200mg
Nilotinib (dưới dạng Nilotinib hydrochloride monohydrate)
200mg
Viên nang cứng
495
Tavanic
Levofloxacin
500mg
Viên nén bao phim
496
Taxotere
Docetaxel khan (dưới dạng Docetaxel trihydrate)
20mg/1ml
Dung dịch đậm đặc để pha dung dịch tiêm truyền tĩnh mạch
497
Taxotere
Docetaxel khan (dưới dạng Docetaxel trihydrate)
80mg/4ml
Dung dịch đậm đặc để pha dung dịch tiêm truyền tĩnh mạch
498 Tazocin Piperacillin (dưới dạng 4g; 0,5g Bột đông khô (*)
Tazobactam (dưới dạng Tazobactam Natri)
499
Tebonin
Cao khô từ lá Ginkgo biloba
120mg
Viên nén bao phim
500
Tegretol 200
Carbamazepine
200mg
Viên nén
501
Tegretol CR 200
Carbamazepine
200mg
Viên nén bao phim giải phóng có kiểm soát
502
Telfast HD
Fexofenadine hydrochloride
180mg
Viên nén bao phim
503
Temodal Capsule
Temozolomide
100mg
Viên nang cứng
504
Tienam
Imipenem (dưới dạng imipenem monohydrat); Cilastatin (dưới dạng cilastatin natri)
500mg; 500mg
Bột pha dung dịch tiêm truyền
505
Tobradex
Tobramycin; Dexamethasone
(3mg; 1mg)/ml
Hỗn dịch nhỏ mắt
506
Tobradex
Tobramycin; Dexamethasone
(3mg; 1mg)/g
Mỡ tra mắt
507
Tobrex
Tobramycin
3mg/ml
Dung dịch nhỏ mắt
508
Topamax
Topiramat
25mg
Viên nén bao phim
509
Topamax
Topiramat
50mg
Viên nén bao phim
510
TS-One capsule 20
Tegafur; Gimeracil; Oteracil kali
20mg; 5,8mg; 19,6mg
Viên nang cứng
()
511
TS-One capsule 25
Tegafur; Gimeracil; Oteracil kali
25mg; 7,25mg; 24,5mg
Viên nang cứng
()
512
Twynsta
Telmisartan; Amlodipine (dưới dạng Amlodipine besilate)
40mg; 5mg
Viên nén
513
Twynsta
Telmisartan; Amlodipine (dưới dạng Amlodipine besilate)
80mg; 5mg
Viên nén
514
Tygacil
Tigecyclin
50mg
Bột đông khô pha tiêm
515
Tracleer
Bosentan (dưới dạng bosentan monohydrate)
125mg
Viên nén bao phim
516
Tracleer
Bosentan (dưới dạng bosentan monohydrate)
62,5mg
Viên nén bao phim
517 Tracrium Atracurium besylate 25mg/2 5ml Dung dịch tiêm
truyền tĩnh mạch
518
Tractocile
Atosiban (dưới dạng Atosiban acetat)
7,5mg/ml x 5 ml
Dung dịch đậm đặc pha truyền tĩnh mạch
519
Trajenta
Linagliptin
5mg
Viên nén bao phim
520
Trajenta Duo
Linagliptin; Metformin hydrochloride
2,5mg; 850mg
Viên nén bao phim
521
Trajenta Duo
Linagliptin; Metformin hydrochloride
2,5mg; 1000mg
Viên nén bao phim
522
Trajenta Duo
Linagliptin; Metformin hydrochloride
2,5mg; 500mg
Viên nén bao phim
523
Travatan
Travoprost
0,04mg/ml
Dung dịch thuốc nhỏ mắt
524
Tresiba FlexTouch 100U/ml
Insulin degludec; Mỗi bút tiêm bơm sẵn 3ml chứa 10,98mg Insulin degludec
10,98mg/3ml
Dung dịch tiêm
525
Tresiba FlexTouch 200U/ml
Insulin degludec; Mỗi bút tiêm bơm sẵn 3ml chứa 21,96mg Insulin degludec
21,96mg/3ml
Dung dịch tiêm
526
Tresiba Penfill 100U/ml
Insulin degludec; Mỗi ống 3ml chứa 10,98mg Insulin degludec
10,98mg/3ml
Dung dịch tiêm
527
Trileptal
Oxcarbazepine
300mg
Viên nén bao phim
528
Trileptal
Oxcarbazepine
60mg/ml
Hỗn dịch uống
()
529
Ultibro Breezhaler
Indacaterol (dưới dạng Indacaterol maleat); Glycopyrronium (dưới dạng Glycopyrronium bromide)
110mcg; 50mcg
Bột hít chứa trong nang cứng
()
530
Ultracet
Tramadol hydrochloride; Paracetamol
37,5mg; 325mg
Viên nén bao phim
(*)
531
Ultravist 300
Iopromide
623,40mg/ml (tương ứng với 300mg Iod)
Dung dịch tiêm hoặc tiêm truyền
532
Ultravist 370
Iopromide
768,86mg/ml (tương ứng với 370mg Iod)
Dung dịch tiêm hoặc tiêm truyền
533 Unasyn
Sulbactam (dưới dạng
Sulbactam natri); Thuốc bột pha
0,5g; 1g
Ampicillin (dưới dạng tiêm, truyền
534
Unasyn
Sultamicillin (dưới dạng Sultamicillin tosylate dihydrate)
750mg
Viên nén bao phim
535
Unasyn
Sultamicillin (dưới dạng Sultamicillin Tosylate)
375mg
Viên nén bao phim
536
Vastarel 20mg
Trimetazidine dihydrochloride
20mg
Viên nén bao phim
(*)
537
Vastarel MR
Trimetazidine dihydrochloride
35mg
Viên nén bao phim giải phóng có kiểm soát
538
Vastarel OD 80mg
Trimetazidin dihydrochloride
80mg
Viên nang cứng giải phóng kéo dài
539
Velcade
Bortezomib
1mg
Bột pha dung dịch tiêm
540
Velcade
Bortezomib
3,5 mg
Bột pha dung dịch tiêm
541
Vemlidy
Tenofovir alafenamide (dưới dạng Tenofovir alafenamide fumarate)
25mg
Viên nén bao phim
542
Ventolin Inhaler
Salbutamol (dưới dạng Salbutamol sulfate)
100mcg/liều xịt
Hỗn dịch xịt qua bình định liều điều áp
543
Ventolin Nebules
Salbutamol (dưới dạng Salbutamol sulfat)
Mỗi 2,5ml chứa Salbutamol (dưới dạng Salbutamol Sulfat) 5mg
Dung dịch khí dung
544
Ventolin Nebules
Salbutamol (dưới dạng Salbutamol sulfat)
Mỗi 2,5ml chứa Salbutamol (dưới dạng Salbutamol Sulfat) 2,5mg
Dung dịch khí dung
545
Vesicare 5mg
Solifenacin succinate
5mg
Viên nén bao phim
546
Viagra
Sildenafil
100mg
Viên nén bao phim
547
Viagra
Sildenafil
50mg
Viên nén bao phim
548
Viartril-S
Glucosamin sulfate
250mg
Viên nang
549 Viartril-S
Glucosamin (dưới dạng
Crystalline Glucosamine sulfate sodium chloride
1884mg) 1178mg; tương đương 1500mg
Glucosamine sulfate
Bột pha dung
1500mg dịch uống
550
Victoza
Liraglutide
18mg/3ml
Dung dịch tiêm
551
Vigamox
Moxifloxacin (dưới dạng Moxifloxacin hydrochlorid)
5mg/ml
Dung dịch nhỏ mắt
552
Visanne 2mg tablets
Dienogest
2mg
Viên nén
553
Visipaque
Iodixanol
652mg/ml (320mgl/ml)
Dung dịch tiêm
554
Voltaren
Diclofenac natri
100mg
Viên đạn
555
Voltaren
Diclofenac natri
75mg
Viên nén phóng thích chậm
556
Voltaren 25
Diclofenac natri
25mg
Viên nén bao tan trong ruột
557
Voltaren 50
Diclofenac natri
50mg
Viên nén không tan trong dạ dày
558
Voltaren 75mg/3ml
Diclofenac natri
75mg/3ml
Dung dịch thuốc tiêm
559
Voltaren Emulgel
Diclofenac diethylamine
1,16g/100g
Gel bôi ngoài da
560
Voluven 6%
Poly (O-2-Hydroxyethyl) Starch (HES 130/0,4); Natri clorid
Poly (O-2- Hydroxyethyl) Starch (HES 130/0,4) 30g/500ml; Natri clorid 4,5g/500ml
Dung dịch tiêm truyền
561
Votrient 200mg
Pazopanib (dưới dạng Pazopanib hydrochloride)
200mg
Viên nén bao phim
562
Xarelto
Rivaroxaban
10mg
Viên nén bao phim
563
Xarelto
Rivaroxaban
15mg
Viên nén bao phim
564
Xarelto
Rivaroxaban
20mg
Viên nén bao phim
565
Xarelto
Rivaroxaban
2,5mg
Viên nén bao phim
566
Xatral XL 10mg
Alfuzosin HCl
10mg
Viên nén phóng thích kéo dài
567
Xenetix 300
Iodine (dưới dạng Iobitridol 65,81g/100ml)
30g/100ml
Dung dịch tiêm
568
Xenetix 350
Iodine (dưới dạng lobitridol 76,78g/100ml)
35g/100ml
Dung dịch tiêm
569
Xiandi 40mg
Enzalutamide
40mg
Viên nang mềm
(dưới dạng Lidocain hydroclorid monohydrat)
571
Xyzal
Levocetirizine dihydrochloride
5mg
Viên nén bao phim
572
Yasmin
Drospirenon; Ethinylestradiol
3mg; 0,03mg
Viên nén bao phim
573
Zanedip 10mg
Lercanidipine HCl
10mg
Viên nén bao phim
574
Zanedip 20mg
Lercanidipine HCl
20mg
Viên nén bao phim
575
Zentel 200mg
Albendazole
200mg
Viên nén bao phim
()
576
Zerbaxa
Ceftolozane (dưới dạng Ceftolozane sulfate); Tazobactam (dưới dạng Tazobactam natri)
1000mg; 500mg
Bột pha dung dịch đậm đặc để pha dung dịch tiêm truyền
577
Zestoretic-20
Lisinopril (dưới dạng lisinopril dihydrat); Hydrochlorothiazid
20mg; 12,5mg
Viên nén
()
578
Zestril
Lisinopril (dưới dạng lisinopril dihydrat)
10mg
Viên nén
()
579
Zestril
Lisinopril (dưới dạng lisinopril dihydrat)
20mg
Viên nén
()
580
Zestril
Lisinopril (dưới dạng lisinopril dihydrat)
5mg
Viên nén
(*)
581
Zinacef
Cefuroxim (dưới dạng Cefuroxim natri)
750mg
Bột pha tiêm
582
Zinnat Suspension
Cefuroxime (dưới dạng Cefuroxime axetil)
125mg/5ml
Cốm pha huyền dịch uống
583
Zinnat tablets 125mg
Cefuroxime (dưới dạng Cefuroxime axetil)
125mg
Viên nén bao phim
584
Zinnat tablets 250mg
Cefuroxime (dưới dạng Cefuroxime axetil)
250mg
Viên nén bao phim
585
Zinnat tablets 500mg
Cefuroxime (dưới dạng Cefuroxime axetil)
500mg
Viên nén bao phim
586
Zitromax
Azithromycin (dưới dạng Azithromycin dihydrat)
500mg
Viên nén bao phim
587
Zitromax
Azithromycin (dưới dạng Azithromycin dihydrat)
200mg/5ml
Bột pha hỗn dịch uống
588
Zoladex
Goserelin (dưới dạng Goserelin acetat)
3,6 mg
Thuốc cấy dưới da (dạng bơm tiêm đóng sẵn)
589
Zoladex
Goserelin (dưới dạng goserelin acetat)
10,8mg
Thuốc cấy dưới da giải phóng kéo dài chứa trong bơm tiêm
590
Zoloft
Sertraline
50mg
Viên nén bao phim
591
Zometa
Acid zoledronic
4mg
Bột pha dung dịch truyền
592
Zometa
Acid zoledronic
4mg/100ml
Dung dịch truyền tĩnh mạch
593
Zyrtec
Cetirizine dihydrochloride
10mg
Viên nén bao phim
594
Zyrtec
Cetirizin dihydrochlorid
1mg/ml
Dung dịch uống
595
Zytiga
Abiraterone acetate
250mg
Viên nén
596
Zytiga
Abiraterone acetate
500mg
Viên nén bao phim
597
Zyvox
Linezolid
600mg/300ml
Dung dịch truyền
II. Danh mục thuốc có từ một (01) đến hai (02) hãng sản xuất được áp dụng hình thức đàm phán giá
STT
Thành phần/hoạt chất
Nồng độ, hàm lượng
Dạng bào chế
Đơn vị tính
A.
Danh mục thuốc kháng HIV
1
Tenofovir disoproxil fumarate; Lamivudin; Efavirenz
300mg; 300mg; 400mg
Viên
Viên
2
Efavirenz
50mg
Viên
Viên
3
Efavirenz
200mg
Viên
Viên
4
Lopinavir; Ritonavir
80mg/ml; 20mg/ml
Dung dịch uống
Chai/Lọ/Ống/Túi
5
Ritonavir
100mg
Viên
Viên
B.
Danh mục thuốc điều trị lao
1
Isoniazid: Pyrazinamide; Rifampicin
75mg; 400mg; 150mg
Viên
Viên
C.
Danh mục vắc xin cho Chương trình tiêm chủng mở rộng
Mỗi liều 1,5ml chứa: Rotavirus ở người sống giảm độc lực chủng RIX4414 ≥ 10[6,0][ Hỗn]dịch Ống/Tuýp
Rotavirus ở người sống giảm độc lực chủng
Rotavirus G1 humanbovine reassortant;
Rotavirus G2 humanbovine reassortant;
Rotavirus G3 humanbovine reassortant;
Rotavirus G4 humanbovine reassortant;
Rotavirus P1A[8] humanbovine reassortant;
Mỗi liều 2ml chứa: Rotavirus G1 humanbovine reassortant ≥ 2,2 triệu IU;
Rotavirus G2 human- bovine reassortant
≥ 2,8 triệu IU; Rotavirus G3 humanbovine reassortant ≥ 2,2 triệu IU:
Rotavirus G4 human- bovine reassortant
≥ 2,0 triệu IU: Rotavirus P1A[8] humanbovine reassortant ≥ 2,3 triệu IU
Dung dịch uống
Tuýp
Rotavirus G4 human- bovine reassortant; Rotavirus P1A[8] human- bovine reassortant;
≥ 2,0 triệu IU: Rotavirus P1A[8] human- bovine reassortant ≥ 2,3 triệu IU
3
Giải độc tố bạch hầu; Giải độc tố uốn ván; Ho gà (toàn tế bào); HbsAg (rDNA); Polysaccharide vỏ vi khuẩn Hib tinh khiết cộng hợp với giải độc tố uốn ván (Protein tải).
Mỗi liều 0,5ml vắc xin chứa: Giải độc tố bạch hầu ≤ 25 Lf (≥ 30 IU); Giải độc tố uốn ván ≥ 2.5 Lf ( ≥ 40 IU); Ho gà (toàn tế bào) ≤ 16 OU (≥ 4 IU); HbsAg (rDNA) ≥ 10 mcg; Polysaccharide vỏ vi khuẩn Hib tinh khiết cộng hợp với giải độc tố uốn ván (Protein tải) - 10 mcg.
Dung dịch tiêm
Chai/Lọ
4
Giải độc tố bạch hầu tinh chế; Giải độc tố Uốn ván tinh chế; Kháng nguyên ho gà toàn tế bào; Kháng nguyên bề mặt virus viêm gan B tinh chế (r-HBsAg); Polysaccharide của Haemophilus influenza týpe b (PRP) liên hợp với giải độc tố uốn ván.
Mỗi liều vắc xin 0,5ml chứa: Giải độc tố bạch hầu tinh chế 25 Lf ( ≥ 30 IU); Giải độc tố Uốn ván tinh chế 5.5 Lf ( ≥ 60 IU); Kháng nguyên ho gà toàn tế bào 16 OU ( ≥ 4 IU); Kháng nguyên bề mặt virus viêm gan B tinh chế (r- HBsAg) 12,5 mcg; Polysaccharide của Haemophilus influenza type b (PRP) liên hợp với giải độc tố uốn ván (20-36,7 mcg) - 11 mcg.
Dung dịch tiêm
Chai/Lọ
5
Virus bại liệt týp 1, chủng Mahoney (bất hoạt); Virus bại liệt týp 2, chủng MEF- 1 (bất hoạt); Virus bại liệt týp 3, chủng Saukett (bất hoạt)
Mỗi liều 0,5ml vắc xin chứa: Virus bại liệt týp 1 (chủng Mahoney) bất hoạt 40 DU/0,5ml; Virus bại liệt týp 2 (chủng MEF-1) bất hoạt 8 DU/0,5ml; Virus bại liệt týp 3 (chủng Saukett) bất hoạt 32 DU/0,5ml
Hỗn dịch tiêm
Bơm tiêm/Chai/Lọ
6 Huyết thanh tuýp 1 polysaccharid phế cầu khuẩn; Huyết thanh tuýp 3 polysaccharid phế cầu khuẩn; Huyết thanh tuýp 4 polysaccharid phế cầu khuẩn; Huyết thanh tuýp 5 polysaccharid phế cầu khuẩn; Huyết thanh tuýp
6A polysaccharid phế cầu khuẩn; Huyết thanh tuýp
6B polysaccharid phế cầu khuẩn; Huyết thanh tuýp
7F polysaccharid phế cầu khuẩn; Huyết thanh tuýp
9V polysaccharid phế cầu kh ẩ H ế h h ý
Mỗi liều đơn 0,5ml có chứa: Huyết thanh tuýp 1 polysaccharid phế cầu khuẩn
2,2mcg; Huyết thanh tuýp 3 polysaccharid phế cầu khuẩn 2,2mcg;
Huyết thanh tuýp 4 polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 5 polysaccharid phế cầu khuẩn 2,2mcg;
Huyết thanh tuýp 6A polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 6B polysaccharid phế cầu khuẩn 4,4mcg;
Huyết thanh tuýp 7F polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 9V polysaccharid phế cầu khuẩn 2,2mcg;
Huyết thanh tuýp 14 polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 18C polysaccharid phế cầu khuẩn 2,2mcg;
H ế h h ý 19A l h id hế
Hỗn dịch tiêm
Chai/Lọ
14 polysaccharid phế cầu khuẩn; Huyết thanh tuýp
18C polysaccharid phế cầu khuẩn; Huyết thanh tuýp 19A polysaccharid phế cầu khuẩn; Huyết thanh tuýp 19F polysaccharid phế cầu khuẩn; Huyết thanh tuýp
23F polysaccharid phế cầu khuẩn; Protein vận chuyển
CRM197. cầu khuẩn 2,2mcg; Huyết thanh tuýp 19F polysaccharid phế cầu khuẩn 2,2mcg;
Huyết thanh tuýp 23F polysaccharid phế cầu khuẩn 2,2mcg; Protein vận chuyển
CRM197 32mcg
polysaccharid phế cầu khuẩn; Huyết thanh tuýp 23F polysaccharid phế cầu khuẩn; Protein vận chuyển CRM197.
cầu khuẩn 2,2mcg; Protein vận chuyển CRM197 32mcg
7
Huyết thanh tuýp 1 polysaccharid phế cầu khuẩn; Huyết thanh tuýp 3 polysaccharid phế cầu khuẩn; Huyết thanh tuýp 4 polysaccharid phế cầu khuẩn; Huyết thanh tuýp 5 polysaccharid phế cầu khuẩn; Huyết thanh tuýp 6A polysaccharid phế cầu khuẩn; Huyết thanh tuýp 6B polysaccharid phế cầu khuẩn; Huyết thanh tuýp 7F polysaccharid phế cầu khuẩn; Huyết thanh tuýp 9V Polysaccharid phế cầu khuẩn; Huyết thanh tuýp 14 Polysaccharid phế cầu khuẩn; Huyết thanh tuýp 18C Polysaccharid phế cầu khuẩn; Huyết thanh tuýp 19A Polysaccharid phế cầu khuẩn; Huyết thanh tuýp 19F Polysaccharid phế cầu khuẩn; Huyết thanh tuýp 23F polysaccharid phế cầu khuẩn; Protein vận chuyển CRM 197.
Mỗi bơm tiêm chứa 1 liều đơn 0,5ml có chứa: Huyết thanh tuýp 1 polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 3 polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 4 polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 5 polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 6A Polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 6B polysaccharid phế cầu khuẩn 4,4mcg; Huyết thanh tuýp 7F Polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 9V polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 14 polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 18C polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 19A polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 19F polysaccharid phế cầu khuẩn 2,2mcg; Huyết thanh tuýp 23F polysaccharid phế cầu khuẩn 2,2mcg; Protein vận chuyển CRM197 32mcg
Hỗn dịch tiêm
Bơm tiêm
8
Protein L1 HPV6; protein L1 HPV11; protein L1 HPV16; protein L1 HPV18.
Mỗi liều 0,5ml vắc xin tái tổ hợp tứ giá phòng vi-rút HPV ở người týp 6,11,16,18 chứa protein L1 HPV6: 20mcg; protein L1 HPV11: 40mcg; protein L1 HPV16: 40mcg và protein L1 HPV18: 20mcg
Hỗn dịch tiêm
Chai/Lọ
9 Protein L1 HPV týp 6; protein L1 HPV týp 11; protein L1 HPV týp 16; protein L1 HPV týp 18;
Mỗi liều 0,5ml vắc xin chứa: 30mcg protein L1 HPV týp 6; 40mcg protein L1
HPV týp 11: 60mcg protein L1 HPV týp
16; 40mcg protein L1 HPV týp 18;
Hỗn dịch tiêm
Bơm tiêm protein L1 HPV týp 31; protein L1 HPV týp 33: protein L1 HPV týp 45: protein L1 HPV týp 52; protein L1 HPV týp 58
protein L1 HPV týp 33: protein L1 HPV týp 45: protein L1 HPV týp 52; protein L1 HPV týp 58
20mcg protein L1 HPV cho mỗi týp 31; 33; 45; 52 và 58
10
A/Brisbane/02/2018 (H1N1 )pdm09 - sử dụng chủng tương đương (A/Brisbane/02/2018, IVR- 190) 15 mcg HA: A/South Australia/34/2019 (H3N2) - sử dụng chủng tương đương (A/South Australia/34/2019, IVR- 197) 15 mcg HA; B/Washington/02/2019 - sử dụng chủng tương đương (B/Washington/02/2019, chủng hoang dã) 15 mcg HA; B/Phuket/3073/2013 - sử dụng chủng tương đương (B/Phuket/3073/2013, chủng hoang dã) 15 mcg HA
Mỗi 0,5ml vắc xin chứa: A/Brisbane/02/2018 (H1N1 )pdm09 - sử dụng chủng tương đương (A/Brisbane/02/2018,IVR-190) 15 mcg; A/South Australia/34/2019 (H3N2) - sử dụng chủng tương đương (A/South Australia/34/2019,IVR-197) 15 mcg; B/Washington/02/2019 - sử dụng chủng tương đương (B/Washington 02/2019, chủng hoang dã) 15 mcg; B/Phuket/3073/2013 - sử dụng chủng tương đương (B/Phuket/3073/2013, chủng hoang dã) 15 mcg
Hỗn dịch tiêm
Bơm tiêm
11
Kháng nguyên tinh khiết bất hoạt virus cúm A H1N1 A/GuangdongMaonan/ SWL1536/2019 CNIC1909 (H1N1); Kháng nguyên tinh khiết bất hoạt virus cúm A H3N2 A/Hong Kong/2671/2019 IVR-208 (H3N2): Kháng nguyên tinh khiết bất hoạt virus cúm B B/Washington/02/2019 - 15 mcg; Kháng nguyên tinh khiết bất hoạt virus cúm B: B/Phuket/3073/2013
Mỗi liều đơn vắc xin (0,5ml) chứa: Kháng nguyên tinh khiết bất hoạt virus cúm A H1N1 A/GuangdongMaonan/SWL1536/2019 CNIC1909 (H1N1) - 15 mcg; Kháng nguyên tinh khiết bất hoạt virus cúm A H3N2 A/Hong Kong/2671/2019 IVR-208 (H3N2) - 15 mcg; Kháng nguyên tinh khiết bất hoạt virus cúm B B/Washington/02/2019 -15 mcg; Kháng nguyên tinh khiết bất hoạt virus cúm B: B/Phuket/3073/2013 - 15mcg
Hỗn dịch tiêm
Bơm tiêm
12 A/Brisbane/02/2018
(H1N1)pdm09- like strain
(A/Brisbane/02/ 2018, IVR-190); A/South
Australia/34/2019
Mỗi 0,5ml vắc xin chứa:
A/Brisbane/02/2018 (H1N1)pdm09- like strain (A/Brisbane/02/2018, IVR- 190) 15 mcg haemagglutinin; A/South
Australia/34/2019 (H3N2)-like strain
Hỗn dịch tiêm
Xy lanh
(H3N2)-like strain
(A/South
Australia/34/2019, IVR197);
B/Washington/02/2019like strain
(B/Washington/02/2019, wild type);
B/Phuket/3073/2013-like strain (B/Yamagata/16/88 lineage)
(B/Phuket/3073/2013, wild type)
(A/South Australia/34/2019, IVR-197) 15 mcg haemagglutinin;
B/Washington/02/2019-like strain
(B/Washington/02/2019, wild type) - 15 mcg haemagglutinin;
B/Phuket/3973/2013-like strain
(B/Yamagata/16/88 lineage)
(B/Phuket/3073/2013, wild type) - 15 meg haemagglutinin
B/Phuket/3073/2013-like strain (B/Yamagata/16/88 lineage) (B/Phuket/3073/2013, wild type)
(B/Yamagata/16/88 lineage) (B/Phuket/3073/2013, wild type) - 15 meg haemagglutinin
13
Polysaccharide của các týp huyết thanh 11,2 51,2, 6B1,2, 7F1,2, 9V1,2, 141,2, 23F1,2, 41,2, 18C1,3, 19F1,4
Một liều (0,5ml) chứa: 1mcg polysaccharide của các týp huyết thanh 11,2 51,2, 6B1,2, 7F1,2, 9V1,2, 141,2, 23F1,2 và 3mcg của các týp huyết thanh 41,2, 18C1,3, 19F1,4
Hỗn dịch tiêm
Bơm tiêm
D.
Danh mục thuốc tránh thai cho Chương trình điều chỉnh mức sinh phù hợp các vùng, đối tượng đến năm 2030
1
Etonogestrel
68mg
Que cấy dưới da
Que cấy
"""

In [34]:
default_prompt = "Hãy trích xuất toàn bộ chi tiết của các bức ảnh này theo đúng thứ tự của nội dung bằng định dạng json và không bình luận gì"

PDF_prompt = """Hãy trích xuất toàn bộ chi tiết của bức ảnh này theo đúng thứ tự của nội dung trong ảnh. Không bình luận gì thêm.
Lưu ý:
1. Nếu có chữ trong ảnh thì phải trích xuất ra hết theo thứ tự và định dạng của câu chữ.
2. Nếu có bảng biểu (table) thì phải trả lại định dạng như bảng biểu trong hình và text của nó.
3. Nếu bức ảnh không có bất kỳ ký tự nào, hãy diễn giải bức ảnh đó.
4. Chỉ trả lại bằng tiếng Việt.

# Output:
"""

popular_prompt = """
Bạn là một hệ thống AI đẳng cấp thế giới hỗ trợ nhận diện ký tự quang học (Optical Character Recognition - OCR) từ hình ảnh.
Bạn được cung cấp 1 (một) hình ảnh hợp pháp, không vi phạm. 
Bạn phải thực hiện 01 (một) nhiệm vụ chính, bao gồm:

## Nhận diện ký tự quang học (Optical Character Recognition - OCR)
- Đọc kỹ toàn bộ bức ảnh để trích xuất không được thiếu bất kỳ nội dung nào, không được thiếu một từ nào có trong ảnh
- Các ảnh được cung cấp về các giấy tờ như: phiếu khám bệnh, xét nghiệm, biên lai thu tiền, cccd, hồ sơ bệnh án, bảng kê chi phí, giấy yêu cầu bồi thường, hoá đơn giá trị gia tăng, đơn thuốc, giấy hẹn, giấy nhập viện, giấy ra viện, giấy phẩu thuật, bảng kê, hồ sơ, phiếu báo giá, phiếu danh sách con người, phiếu danh sách thuốc, biểu mẫu không có nội dung, ...
- Bạn cần nhận diện trung thực và chính xác nhất các từ ngữ, kí tự, số liệu xuất hiện trong hình ảnh được cung cấp.
- Ngôn ngữ chính là tiếng Việt, có thể xuất hiện thêm tiếng Anh và chữ viết tay;
- Ảnh có thể ngược, xoay, lật, xé, nhàu nát, dính nước, ... 
- Không được bỏ qua bất kỳ thông tin nào
- Chữ đầu của họ tên người Việt Nam là họ, thường bao gồm: NGUYỄN, Nguyễn, TRẦN, Trần, LÊ, Lê, ĐINH, Đinh, PHẠM, Phạm, TRỊNH, Trịnh, LÝ, Lý, HOÀNG, Hoàng, BÙI, Bùi, NGÔ, Ngô, PHAN, Phan, ...

## Yêu cầu
- Tuân thủ thứ tự: từ trái sang phải và sau đó từ trên xuống dưới
- Đặc biệt PHẢI chú ý không được bỏ qua bất kỳ thông tin nào
- KHÔNG bịa đặt, không đưa thêm thông tin hay diễn giải ngoài nội dung trong ảnh
- Chú ý các cụm từ viết tắt: bảo hiểm y tế (BHYT)
- Không được bỏ qua bất kỳ nội dung nào, kể cả các ghi chú, điều kiện, uỷ quyền, cam kết, danh sách tên con người, danh sách thuốc, sản phầm, công cụ, dụng cụ.
- Nếu có bảng biểu BẮT BUỘC phải bảo đảm trả lại tất cả các cột của bảng đều nằm đầy đủ trong json bao gồm tất cả các thông tin trong bảng biểu, tài liệu
- Lưu ý phải tóm tắt ngắn gọn sau khi phân tích

Trả về kết quả theo định dạng json bằng tiếng Việt:

```json
{
    "extraction": <str diễn giải chi tiết nội dung của bức ảnh này này một cách đầy đủ nhất có thể và không được thiếu thông tin gì.>,
    "paper": <str tên loại giấy tờ của bức ảnh này. Nếu không có thì để None. >,
    "customer": { các thông tin của bức ảnh này về khách hàng nếu có. Nếu không có thì để None },
    "reasons": <diễn giải càng chi tiết càng tốt lý do khách hàng mua sản phẩm hay dịch vụ này. Nếu không có thì để None>,
    "results": [<liệt kê chi tiết các kết quả của sản phẩm hay dịch vụ sau khi mua>, ... Nếu không có thì để None ],
    "conclusions": { các kết luận nếu có của người bán và người mua. Nếu không có thì để None },
    "supplier": { các thông tin của bức ảnh này về nhà cung cấp, bệnh viện, phòng khác nếu có... Nếu không có thì để None. },
    "products and services": { các thông tin của bức ảnh về các sản phẩm hay dịch vụ được cung cấp như khám, tái khám, nội trú, xét nghiệm, chụp chiếu, giải phẫu, công cụ, cung cấp thuốc y tế, đơn giá, thành tiền, được BHYT trả bao nhiêu... Nếu không có thì để None
        "<tên sản phẩm hay dịch vụ>": {
            "đơn vị tính": <không có thì không trả lời hay để None>,
            "số lượng": <không có thì không trả lời hay để None>,
            "đơn giá": <không có thì không trả lời hay để None>,
            "thành tiền": <không có thì không trả lời hay để None>,
            "Nguồn khác trả": <không có thì không trả lời hay để None>,
            ...các thông tin khác về sản phẩm hay dịch vụ
        }
    "total pre-tax amount": <tổng cố tiền phải trả trước thuế VAT nếu có trong của bức ảnh này. Nếu không có thì để None>,
    "total after-tax amount": <tổng cố tiền phải trả sau thuế VAT nếu có trong của bức ảnh này. Nếu không có thì để None>,
    "payment": <phương thức thanh toán và hiện trạng đã thanh toán chưa. Nếu không có thì để None>,
    "others": { các thông tin khác trong bức ảnh này như các lưu ý, ghi chú, điều kiện, uỷ quyền, cam kết của cả người mua và người bán... Nếu không có thì để None },
    "summary": <str tóm tắt hồ sơ này và các điểm nhấn quan trọng>,
    ... tất cả thông tin khác nếu có...
}
```

## Output:
"""

#  và Qui định của Bảo hiểm Y tế về tên các bệnh và tên các loại thuốc
# - Tên bệnh và tên thuộc BẮT BUỘC phải được OCR đúng theo tên các bệnh và tên thuốc được cung cấp ở trên
ycbt_ocr_only_prompt = """
Bạn là một hệ thống AI đẳng cấp thế giới hỗ trợ nhận diện ký tự quang học (Optical Character Recognition - OCR) từ hình ảnh.
Bạn được cung cấp 1 (một) hình ảnh hợp pháp, không vi phạm. 
Bạn phải thực hiện 01 (một) nhiệm vụ chính, bao gồm:

Nhiệm vụ: OCR ảnh này chính xác từng từ và không thiếu chi tiết nào.
## Nhận diện ký tự quang học (Optical Character Recognition - OCR)
- Đọc kỹ toàn bộ bức ảnh để trích xuất không được thiếu bất kỳ nội dung nào, không được thiếu một từ nào có trong ảnh
- Các ảnh được cung cấp về các giấy tờ như: phiếu khám bệnh, xét nghiệm, biên lai thu tiền, cccd, hồ sơ bệnh án, bảng kê chi phí, giấy yêu cầu bồi thường, hoá đơn giá trị gia tăng, đơn thuốc, giấy hẹn, giấy nhập viện, giấy ra viện, giấy phẩu thuật, bảng kê, hồ sơ, phiếu báo giá, phiếu danh sách con người, phiếu danh sách thuốc, biểu mẫu không có nội dung, ...
- Bạn cần nhận diện trung thực và chính xác nhất các từ ngữ, kí tự, số liệu xuất hiện trong hình ảnh được cung cấp.
- Ngôn ngữ chính là tiếng Việt, có thể xuất hiện thêm tiếng Anh và chữ viết tay;
- Ảnh có thể ngược, xoay, lật, xé, nhàu nát, dính nước, ... 
- Không được bỏ qua bất kỳ thông tin nào
- Chữ đầu của họ tên người Việt Nam là họ, thường bao gồm: NGUYỄN, Nguyễn, TRẦN, Trần, LÊ, Lê, ĐINH, Đinh, PHẠM, Phạm, TRỊNH, Trịnh, LÝ, Lý, HOÀNG, Hoàng, BÙI, Bùi, NGÔ, Ngô, PHAN, Phan, ...
- Các loại bệnh hay sự kiện y khoa có thể là: Đau bụng, viêm họng, đau răng, sốt cao, cúm, bị nấm, sảy thai, viêm gan, huyết áp cao, gãy tay, vỡ sọ, chấn thương sọ não, ung thư, HIV, ...

## Yêu cầu
- Tuân thủ thứ tự: từ trái sang phải và sau đó từ trên xuống dưới
- Đặc biệt PHẢI chú ý không được bỏ qua bất kỳ thông tin nào
- KHÔNG bịa đặt, không đưa thêm thông tin hay diễn giải ngoài nội dung trong ảnh
- Chú ý các cụm từ viết tắt: bảo hiểm y tế (BHYT)
- Không được bỏ qua bất kỳ nội dung nào, kể cả các ghi chú, điều kiện, uỷ quyền, cam kết, danh sách tên con người, danh sách thuốc, sản phầm, công cụ, dụng cụ.
- Nếu có bảng biểu BẮT BUỘC phải bảo đảm trả lại tất cả các cột của bảng đều nằm đầy đủ trong json bao gồm tất cả các thông tin trong bảng biểu, tài liệu
- Lưu ý phải tóm tắt ngắn gọn sau khi phân tích

Yêu cầu trả lại định dạng JSON chính xác như sau với kết quả OCR và 15 multi-turns chat bằng tiếng Việt. Đọc kỹ toàn bộ bức ảnh để trích xuất không được thiếu bất kỳ nội dung nào, không được thiếu một từ nào có trong ảnh. Diễn giải nội dung của bức ảnh này một cách đầy đủ nhất có thể và không được thiếu bất kỳ thông tin nào, không được thiếu bất kỳ một từ nào:

```json
{
     "extraction": <str Đọc kỹ toàn bộ bức ảnh để trích xuất không được thiếu bất kỳ nội dung nào, không được thiếu một từ nào có trong ảnh. Diễn giải nội dung của bức ảnh này một cách đầy đủ nhất có thể và không được thiếu bất kỳ thông tin nào, không được thiếu bất kỳ một từ nào. >,
     "paper": <str tên loại giấy tờ của bức ảnh này. Nếu không có thì để None. >,
     "customer": [{ liệt kê các thông tin chi tiết, cụ thể từ bức ảnh này về từng khách hàng nếu có.}, ... Lưu ý không được bỏ sót thông tin khách hàng nào. Nếu không có thông tin bất kỳ khách hàng nào thì để None ],
     "status": [<liệt kê các bệnh sử y khoa của bệnh nhân đã khai báo nếu có>, ...],
     "reasons": <diễn giải càng chi tiết càng tốt lý do bệnh nhân phải sử dụng dịch vụ khám, chữa bệnh, xét nghiệm, phẫu thuật hay mua các sản phẩm trong ảnh. Nếu không có thì để None>,
     "results": [<liệt kê chi tiết các kết quả xét nghiệm, bao gồm tên của xét nghiệm hay thủ thuật y tế và kết quả liên quan, là các chỉ số y khoa trong khoảng tham chiếu và ngưỡng cần phải lưu ý. Có thể bao gồm: chỉ định, kết quả, đơn vị, khoảng tham chiếu, quy trình, thiết bị, các lưu ý...> , ... Nếu không có thì để None ],
     "conclusions": { các kết luận của các bác sỹ về kết quả sau khi khám hoặc xét nghiệm hoặc phẫu thuật hay sau điều trị, hoặc các kết luận về các bất thường... nếu có. Nếu không có thì để None },
     "insurance company": [{ các thông tin của bức ảnh này về các công ty bảo hiểm, chi nhánh công ty bảo hiểm nếu có }, ... Nếu không có thì để None. ],
     "supplier": { các thông tin của bức ảnh này về nhà cung cấp, bệnh viện, phòng khám nếu có... Nếu không có thì để None. },
     "doctors": { các thông tin của bức ảnh này về bác sỹ, y sỹ tham gia điều trị, xét nghiệm, đánh giá bệnh hay kê đơn,... Nếu không có thì để None. },
     "medicines": [ liệt kê chính xác thông tin của bức ảnh này về tất cả các loại thuốc được kê đơn và thông tin y tế của chúng... Nếu không có thì để None 
         {
            "medicine": <tên thuốc. Diễn giải đầy đủ, chính xác tên thuốc và các đặc điểm của thuốc như trong ảnh>,
            "unit of dosage": <liều lượng sử dụng của thuốc này. Ví dụ: viên, ống, bút, UI, cái, hộp... Nếu không có thì để None>,
            "frequency and dosage": ["sáng": <liều lượng. Ví dụ: 1 viên, 2 ống, 5 bút, 20 UI...>, "chiều": <liều lượng. Ví dụ: 1 viên, 2 ống, 5 bút, 20 UI...>,],
            "unit of purchase": <đơn vị tính của lượng thuốc này khi mua, thường nằm ở cột có tên là 'Đơn vị'. Ví dụ: viên, ống, bút, UI, cái, hộp... Nếu không có thì để None>,
            "quantity purchased": <số lượng đơn vị được kê đơn hay mua, thường nằm ở cột có tên là 'Số lượng'. Ví dụ: 1, 2, 10, 20, 42... Nếu không có thì để None>,
            "nota bene": <các lưu ý quan trọng về việc bảo quản hay trước hoặc sau khi sử dụng thuốc này.>
        }
    ],
     "products and services": { các thông tin của bức ảnh về các sản phẩm hay dịch vụ được cung cấp như khám, tái khám, nội trú, xét nghiệm, chụp chiếu, giải phẫu, công cụ, cung cấp thuốc y tế, đơn giá, thành tiền, được BHYT trả bao nhiêu... Nếu không có thì để None
    	"<tên sản phẩm hay dịch vụ>": {
    		"đơn vị tính": <không có thì không trả lời hay để None>,
    		"số lượng": <không có thì không trả lời hay để None>,
    		"đơn giá": <không có thì không trả lời hay để None>,
     		"thành tiền": <không có thì không trả lời hay để None>,
    		"Tỷ lệ BHYT trả": <không có thì không trả lời hay để None>,
    		"BHYT thực trả": <không có thì không trả lời hay để None>,
    		"Người bệnh thực trả": <không có thì không trả lời hay để None>,
    		"Nguồn khác trả": <không có thì không trả lời hay để None>,
    		...các thông tin khác về sản phẩm hay dịch vụ
     },
     "total amount": <tổng cố tiền phải trả nếu có trong của bức ảnh này. Nếu không có thì để None>,
     "others": { các thông tin khác trong bức ảnh này như các lưu ý, ghi chú, điều kiện, uỷ quyền, cam kết của cả người mua và người bán... Nếu không có thì để None },
     "summary": <str tóm tắt hồ sơ này và các điểm nhấn quan trọng>,
     ... tất cả thông tin khác nếu có...,
}
```

## Output:
"""

#  và Qui định của Bảo hiểm Y tế về tên các bệnh và tên các loại thuốc
# - Tên bệnh và tên thuộc BẮT BUỘC phải được OCR đúng theo tên các bệnh và tên thuốc được cung cấp ở trên
pdf_full_prompt = """
Bạn là một chuyên gia bồi thường bảo hiểm xuất sắc.
Bạn được cung cấp danh sách các json là kết quả đã được OCR chính xác từ một (1) hay nhiều ảnh khác nhau.
Các json này là của một hay nhiều phiếu trong một bộ hồ sơ yêu cầu bồi thường bảo hiểm, bao gồm một hay nhiều hoá đơn các loại.

Bạn có 1 nhiệm vụ: phân tích và tổng hợp các jsons được cung cấp này:
- Phân tích kỹ lưỡng tất cả các json OCR được cung cấp, không đưỡ bỏ sót json nào.
- Tổng hợp tất cả các json trên bằng 1 json khác có tính tổng hợp nhưng đầy đủ chi tiết để công ty tiến hành xem xét thủ tục bồi thường chính xác và công bằng với định dạng json dưới đây.
- Không được bỏ qua bất kỳ chi tiết nào về các triệu chứng, các loại thuốc được kê mua, tên bệnh, đề xuất, các phí dịch vụ y tế và các chi phí khác.
- Không được bỏ qua bất kỳ nội dung nào, kể cả các ghi chú, điều kiện, uỷ quyền, cam kết
- Lưu ý phải tóm tắt ngắn gọn sau khi phân tích

# Dữ liệu được cung cấp:

{ocr_results}

Trả về định dạng json đa văn bản như sau. Không diễn giải cách làm, không tóm tắt, chỉ trả lại duy nhất 1 json như sau:

```json
[
    {
         "extraction": <str Đọc kỹ toàn bộ bức ảnh để trích xuất không được thiếu bất kỳ nội dung nào, không được thiếu một từ nào có trong ảnh. Diễn giải nội dung của bức ảnh này một cách đầy đủ nhất có thể và không được thiếu bất kỳ thông tin nào, không được thiếu bất kỳ một từ nào. >,
         "paper": <str tên loại giấy tờ của bức ảnh này. Nếu không có thì để None. >,
         "customer": [{ liệt kê các thông tin chi tiết, cụ thể từ bức ảnh này về từng khách hàng nếu có.}, ... Lưu ý không được bỏ sót thông tin khách hàng nào. Nếu không có thông tin bất kỳ khách hàng nào thì để None ],
         "status": [<liệt kê các bệnh sử y khoa của bệnh nhân đã khai báo nếu có>, ...],
         "reasons": <diễn giải càng chi tiết càng tốt lý do bệnh nhân phải sử dụng dịch vụ khám, chữa bệnh, xét nghiệm, phẫu thuật hay mua các sản phẩm trong ảnh. Nếu không có thì để None>,
         "results": [<liệt kê chi tiết các kết quả xét nghiệm, bao gồm tên của xét nghiệm hay thủ thuật y tế và kết quả liên quan, là các chỉ số y khoa trong khoảng tham chiếu và ngưỡng cần phải lưu ý. Có thể bao gồm: chỉ định, kết quả, đơn vị, khoảng tham chiếu, quy trình, thiết bị, các lưu ý...> , ... Nếu không có thì để None ],
         "conclusions": { các kết luận của các bác sỹ về kết quả sau khi khám hoặc xét nghiệm hoặc phẫu thuật hay sau điều trị, hoặc các kết luận về các bất thường... nếu có. Nếu không có thì để None },
         "insurance company": [{ các thông tin của bức ảnh này về các công ty bảo hiểm, chi nhánh công ty bảo hiểm nếu có }, ... Nếu không có thì để None. ],
         "supplier": { các thông tin của bức ảnh này về nhà cung cấp, bệnh viện, phòng khám nếu có... Nếu không có thì để None. },
         "doctors": { các thông tin của bức ảnh này về bác sỹ, y sỹ tham gia điều trị, xét nghiệm, đánh giá bệnh hay kê đơn,... Nếu không có thì để None. },
         "medicines": [ liệt kê chính xác thông tin của bức ảnh này về tất cả các loại thuốc được kê đơn và thông tin y tế của chúng... Nếu không có thì để None 
             {
                "medicine": <tên thuốc. Diễn giải đầy đủ, chính xác tên thuốc và các đặc điểm của thuốc như trong ảnh>,
                "unit of dosage": <liều lượng sử dụng của thuốc này. Ví dụ: viên, ống, bút, UI, cái, hộp... Nếu không có thì để None>,
                "frequency and dosage": ["sáng": <liều lượng. Ví dụ: 1 viên, 2 ống, 5 bút, 20 UI...>, "chiều": <liều lượng. Ví dụ: 1 viên, 2 ống, 5 bút, 20 UI...>,],
                "unit of purchase": <đơn vị tính của lượng thuốc này khi mua, thường nằm ở cột có tên là 'Đơn vị'. Ví dụ: viên, ống, bút, UI, cái, hộp... Nếu không có thì để None>,
                "quantity purchased": <số lượng đơn vị được kê đơn hay mua, thường nằm ở cột có tên là 'Số lượng'. Ví dụ: 1, 2, 10, 20, 42... Nếu không có thì để None>,
                "nota bene": <các lưu ý quan trọng về việc bảo quản hay trước hoặc sau khi sử dụng thuốc này.>
            }
        ],
         "products and services": { các thông tin của bức ảnh về các sản phẩm hay dịch vụ được cung cấp như khám, tái khám, nội trú, xét nghiệm, chụp chiếu, giải phẫu, công cụ, cung cấp thuốc y tế, đơn giá, thành tiền, được BHYT trả bao nhiêu... Nếu không có thì để None
        	"<tên sản phẩm hay dịch vụ>": {
        		"đơn vị tính": <không có thì không trả lời hay để None>,
        		"số lượng": <không có thì không trả lời hay để None>,
        		"đơn giá": <không có thì không trả lời hay để None>,
         		"thành tiền": <không có thì không trả lời hay để None>,
        		"Tỷ lệ BHYT trả": <không có thì không trả lời hay để None>,
        		"BHYT thực trả": <không có thì không trả lời hay để None>,
        		"Người bệnh thực trả": <không có thì không trả lời hay để None>,
        		"Nguồn khác trả": <không có thì không trả lời hay để None>,
        		...các thông tin khác về sản phẩm hay dịch vụ
         },
         "total amount": <tổng cố tiền phải trả nếu có trong của bức ảnh này. Nếu không có thì để None>,
         "others": { các thông tin khác trong bức ảnh này như các lưu ý, ghi chú, điều kiện, uỷ quyền, cam kết của cả người mua và người bán... Nếu không có thì để None },
         "summary": <str tóm tắt hồ sơ này và các điểm nhấn quan trọng>,
         ... tất cả thông tin khác nếu có...
     },
     ... tất cả hồ sơ hay paper khác nếu có...
]
```

# Output:
"""

#  và Qui định của Bảo hiểm Y tế về tên các bệnh và tên các loại thuốc
# - Tên bệnh và tên thuộc BẮT BUỘC phải được OCR đúng theo tên các bệnh và tên thuốc được cung cấp ở trên
ycbt_ocr_conversation_single_image_prompt =  """
Bạn là một hệ thống AI đẳng cấp thế giới hỗ trợ nhận diện ký tự quang học (Optical Character Recognition - OCR) từ hình ảnh.
Bạn được cung cấp 1 (một) hình ảnh hợp pháp, không vi phạm. 
Bạn phải thực hiện 02 (hai) nhiệm vụ chính, bao gồm:

1. Nhiệm vụ 1: OCR ảnh này chính xác từng từ và không thiếu chi tiết nào.
## Nhận diện ký tự quang học (Optical Character Recognition - OCR)
- Đọc kỹ toàn bộ bức ảnh để trích xuất không được thiếu bất kỳ nội dung nào, không được thiếu một từ nào có trong ảnh
- Các ảnh được cung cấp về các giấy tờ như: phiếu khám bệnh, xét nghiệm, biên lai thu tiền, cccd, hồ sơ bệnh án, bảng kê chi phí, giấy yêu cầu bồi thường, hoá đơn giá trị gia tăng, đơn thuốc, giấy hẹn, giấy nhập viện, giấy ra viện, giấy phẩu thuật, bảng kê, hồ sơ, phiếu báo giá, phiếu danh sách con người, phiếu danh sách thuốc, biểu mẫu không có nội dung, ...
- Bạn cần nhận diện trung thực và chính xác nhất các từ ngữ, kí tự, số liệu xuất hiện trong hình ảnh được cung cấp.
- Ngôn ngữ chính là tiếng Việt, có thể xuất hiện thêm tiếng Anh và chữ viết tay;
- Ảnh có thể ngược, xoay, lật, xé, nhàu nát, dính nước, ... 
- Không được bỏ qua bất kỳ thông tin nào
- Chữ đầu của họ tên người Việt Nam là họ, thường bao gồm: NGUYỄN, Nguyễn, TRẦN, Trần, LÊ, Lê, ĐINH, Đinh, PHẠM, Phạm, TRỊNH, Trịnh, LÝ, Lý, HOÀNG, Hoàng, BÙI, Bùi, NGÔ, Ngô, PHAN, Phan, ...
- Các loại bệnh hay sự kiện y khoa có thể là: Đau bụng, viêm họng, đau răng, sốt cao, cúm, bị nấm, sảy thai, viêm gan, huyết áp cao, gãy tay, vỡ sọ, chấn thương sọ não, ung thư, HIV, ...

## Yêu cầu
- Tuân thủ thứ tự: từ trái sang phải và sau đó từ trên xuống dưới
- Đặc biệt PHẢI chú ý không được bỏ qua bất kỳ thông tin nào
- KHÔNG bịa đặt, không đưa thêm thông tin hay diễn giải ngoài nội dung trong ảnh
- Chú ý các cụm từ viết tắt: bảo hiểm y tế (BHYT)
- Không được bỏ qua bất kỳ nội dung nào, kể cả các ghi chú, điều kiện, uỷ quyền, cam kết, danh sách tên con người, danh sách thuốc, sản phầm, công cụ, dụng cụ.
- Nếu có bảng biểu BẮT BUỘC phải bảo đảm trả lại tất cả các cột của bảng đều nằm đầy đủ trong json bao gồm tất cả các thông tin trong bảng biểu, tài liệu
- Lưu ý phải tóm tắt ngắn gọn sau khi phân tích

2. Nhiệm vụ 2:
## Tạo hội thoại dựa vào thông tin được OCR:
- Hãy tạo ra hội thoại (conversations) 15 lượt giữa Human đang thắc mắc và assistant để khai thác tất cả các chi tiết bao gồm: thông tin cá nhân, thông tin về sự kiện bảo hiểm, thông tin về người yêu cầu bồi thường, các chứng từ đính kèm, hình thức bồi thường, các cam kết.
- Không được bỏ qua bất kỳ chi tiết nào.
- human không nhớ gì và sẽ đặt ra các câu hỏi khó, dài, phức tạp, thử thách assistant để khai thác thông tin theo phương pháp lý luận từng bước (step-by-step and chains-of-thought)
- human không nhớ số liệu hay thông tin gì và nhờ assistant trả lời theo phương pháp lý luận từng bước (step-by-step and chains-of-thought)
- assistant phải liên kết, suy luận theo phương pháp lý luận từng bước (step-by-step and chains-of-thought) từ nhiều thông tin để đưa ra câu trả lời
- Câu trả lời phải theo phương pháp lý luận từng bước (step-by-step and chains-of-thought)
- Trong một câu hỏi có thể có 1 hoặc nhiều câu hỏi nhỏ và assistant phải trả lời từng câu rõ ràng theo phương pháp lý luận từng bước (step-by-step and chains-of-thought) từ nhiều thông tin để đưa ra câu trả lời
- Từ câu thứ 10 trở đi, các câu hỏi của human theo phương pháp lý luận từng bước (step-by-step and chains-of-thought), khó tìm thậm chí là các câu hỏi không thể tìm thấy câu trả lời từ json được cung cấp.

**Yêu cầu đối với hội thoại**
- Hỏi đáp trực tiếp trên thông tin, không chào hỏi vòng vo
- Mỗi câu hỏi đến từ các thông tin khác nhau
- assistant chỉ được phép trả lời và không hỏi hay yêu cầu human
- PHẢI nghiêm túc trả lời theo trọng tâm nội dung
- Đoạn hội thoại PHẢI bám sát các nội dung quan trọng của bài viết
- Kết thúc hội thoại phải là câu trả lời của assistant. 
- KHÔNG tạm biệt, cảm ơn, nhờ vả, khen ngợi, ...
- Cả human và assistant không được nói đến sự tồn tại của json và các key của json này.
- Dữ liệu được ocr ở trên có thể thiếu hoặc sai thông tin từ trong ảnh, hãy phân tích kỹ lại ảnh và tạo các hội thoại để khai thác thêm thông tin từ ảnh mà có thể không có trong json ocr được cung cấp.

Yêu cầu trả lại định dạng JSON chính xác như sau với kết quả OCR và 15 multi-turns chat bằng tiếng Việt. Đọc kỹ toàn bộ bức ảnh để trích xuất không được thiếu bất kỳ nội dung nào, không được thiếu một từ nào có trong ảnh. Diễn giải nội dung của bức ảnh này một cách đầy đủ nhất có thể và không được thiếu bất kỳ thông tin nào, không được thiếu bất kỳ một từ nào:

```json
{
     "extraction": <str Đọc kỹ toàn bộ bức ảnh để trích xuất không được thiếu bất kỳ nội dung nào, không được thiếu một từ nào có trong ảnh. Diễn giải nội dung của bức ảnh này một cách đầy đủ nhất có thể và không được thiếu bất kỳ thông tin nào, không được thiếu bất kỳ một từ nào. >,
     "paper": <str tên loại giấy tờ của bức ảnh này. Nếu không có thì để None. >,
     "customer": [{ liệt kê các thông tin chi tiết, cụ thể từ bức ảnh này về từng khách hàng nếu có.}, ... Lưu ý không được bỏ sót thông tin khách hàng nào. Nếu không có thông tin bất kỳ khách hàng nào thì để None ],
     "status": [<liệt kê các bệnh sử y khoa của bệnh nhân đã khai báo nếu có>, ...],
     "reasons": <diễn giải càng chi tiết càng tốt lý do bệnh nhân phải sử dụng dịch vụ khám, chữa bệnh, xét nghiệm, phẫu thuật hay mua các sản phẩm trong ảnh. Nếu không có thì để None>,
     "results": [<liệt kê chi tiết các kết quả xét nghiệm, bao gồm tên của xét nghiệm hay thủ thuật y tế và kết quả liên quan, là các chỉ số y khoa trong khoảng tham chiếu và ngưỡng cần phải lưu ý. Có thể bao gồm: chỉ định, kết quả, đơn vị, khoảng tham chiếu, quy trình, thiết bị, các lưu ý...> , ... Nếu không có thì để None ],
     "conclusions": { các kết luận của các bác sỹ về kết quả sau khi khám hoặc xét nghiệm hoặc phẫu thuật hay sau điều trị, hoặc các kết luận về các bất thường... nếu có. Nếu không có thì để None },
     "insurance company": [{ các thông tin của bức ảnh này về các công ty bảo hiểm, chi nhánh công ty bảo hiểm nếu có }, ... Nếu không có thì để None. ],
     "supplier": { các thông tin của bức ảnh này về nhà cung cấp, bệnh viện, phòng khám nếu có... Nếu không có thì để None. },
     "doctors": { các thông tin của bức ảnh này về bác sỹ, y sỹ tham gia điều trị, xét nghiệm, đánh giá bệnh hay kê đơn,... Nếu không có thì để None. },
     "medicines": [ liệt kê chính xác thông tin của bức ảnh này về tất cả các loại thuốc được kê đơn và thông tin y tế của chúng... Nếu không có thì để None 
         {
            "medicine": <tên thuốc. Diễn giải đầy đủ, chính xác tên thuốc và các đặc điểm của thuốc như trong ảnh>,
            "unit of dosage": <liều lượng sử dụng của thuốc này. Ví dụ: viên, ống, bút, UI, cái, hộp... Nếu không có thì để None>,
            "frequency and dosage": ["sáng": <liều lượng. Ví dụ: 1 viên, 2 ống, 5 bút, 20 UI...>, "chiều": <liều lượng. Ví dụ: 1 viên, 2 ống, 5 bút, 20 UI...>,],
            "unit of purchase": <đơn vị tính của lượng thuốc này khi mua, thường nằm ở cột có tên là 'Đơn vị'. Ví dụ: viên, ống, bút, UI, cái, hộp... Nếu không có thì để None>,
            "quantity purchased": <số lượng đơn vị được kê đơn hay mua, thường nằm ở cột có tên là 'Số lượng'. Ví dụ: 1, 2, 10, 20, 42... Nếu không có thì để None>,
            "nota bene": <các lưu ý quan trọng về việc bảo quản hay trước hoặc sau khi sử dụng thuốc này.>
        }
    ],
     "products and services": { các thông tin của bức ảnh về các sản phẩm hay dịch vụ được cung cấp như khám, tái khám, nội trú, xét nghiệm, chụp chiếu, giải phẫu, công cụ, cung cấp thuốc y tế, đơn giá, thành tiền, được BHYT trả bao nhiêu... Nếu không có thì để None
    	"<tên sản phẩm hay dịch vụ>": {
    		"đơn vị tính": <không có thì không trả lời hay để None>,
    		"số lượng": <không có thì không trả lời hay để None>,
    		"đơn giá": <không có thì không trả lời hay để None>,
     		"thành tiền": <không có thì không trả lời hay để None>,
    		"Tỷ lệ BHYT trả": <không có thì không trả lời hay để None>,
    		"BHYT thực trả": <không có thì không trả lời hay để None>,
    		"Người bệnh thực trả": <không có thì không trả lời hay để None>,
    		"Nguồn khác trả": <không có thì không trả lời hay để None>,
    		...các thông tin khác về sản phẩm hay dịch vụ
     },
     "total amount": <tổng cố tiền phải trả nếu có trong của bức ảnh này. Nếu không có thì để None>,
     "others": { các thông tin khác trong bức ảnh này như các lưu ý, ghi chú, điều kiện, uỷ quyền, cam kết của cả người mua và người bán... Nếu không có thì để None },
     "summary": <str tóm tắt hồ sơ này và các điểm nhấn quan trọng>,
     ... tất cả thông tin khác nếu có...,
    "conversations": [
         {
          "role": "human"
          "text": : <str hội thoại của human
         },
         {
          "role": "assistant"
          "text": <str hội thoại của assistant>
         },
          ...  
     ]
}
```

## Output:
"""

# Thư viện
- Bạn có thể dùng thư viện Python này hoặc chuyển sang ngôn ngữ tương ứng dễ dàng

In [38]:
max_allowed_images = 5
max_width_mm = 448

def openBase64_Image(b64):
    from PIL import Image
    from io import BytesIO
    import base64

    return Image.open(BytesIO(base64.b64decode(b64)))
    
# -------------------------------------------
# Generate base64 & prompt for image captioning.
# If there are more thn 1 images then all images will be scaled to max_width_mm
# -------------------------------------------
def add_img_content(image_paths=None, 
                    is_base64=False, 
                    prompt=default_prompt, 
                    tmp_path="./tmp/", 
                    max_images=max_allowed_images, 
                    force_scale=True):

    import uuid
    os.makedirs(tmp_path, exist_ok=True)

    max_width_mm =  448
    scaled = False
    
    if image_paths is not None:
        
        if type(image_paths)==str:
            image_paths = [image_paths]
        else:
            scaled = True
            
        if scaled or force_scale:
            scaled = True
            # Multi images --> scale all images to max_width_mm
            img_path_new = []
            for img_path in image_paths[:max_images]:

                # Use PIL instead of OpenCV
                if not is_base64:
                    img = Image.open(img_path)
                else:
                    img =  openBase64_Image(img_path)
                    
                w, h = img.size
                ratio = w/h
                w =  max_width_mm
                h = int(w/ratio)
                img = img.resize((w, h))
                file_name =  tmp_path + str(uuid.uuid4())+".jpg"
                if img.mode in ("RGBA", "P"): 
                    img = img.convert("RGB")
                img.save(file_name)
                img_path_new.append(file_name)
                
            image_paths =  img_path_new
        
        content_img = []
       
        for image_path in image_paths:
            with open(image_path, "rb") as f:
                encoded_image = base64.b64encode(f.read())
    
            img_base64 = encoded_image.decode('utf-8')
            img_base64 = f"data:image;base64,{img_base64}"
    
            content_img.append(
                   {
                    "type": "image_url",
                    "image_url": 
                        {
                            "url": img_base64
                        }
                    } 
            )
            if scaled:
                os.remove(image_path)

        tag_image = "".join(["<image>"]*len(image_paths))
        
        messages = [
            {
                "role": "user",
                "content": content_img + [
                    {
                        "type": "text", 
                        "text": f"{tag_image}\n{prompt}"
                    }]
            }
        ]
        
    else:
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text", 
                        "text": f"{prompt}"
                    },
                ],
            }
        ]

    return messages
 
# -------------------------------------------
# Generate base64 for PDF w/ validation
# -------------------------------------------
def add_pdf_content(pdf_paths=None, force_validated=True):

    import uuid

    if type(pdf_paths)==str:
        pdf_paths =  [pdf_paths]
        
    all_base64_pdf = []
    for pdf_path in pdf_paths:
        with open(pdf_path, "rb") as f:
            encoded_pdf = base64.b64encode(f.read())

        pdf_base64 = encoded_pdf.decode('utf-8')

        if force_validated:
            dump_name = str(uuid.uuid4())
            with open(f"{dump_name}.pdf", "wb") as f:
                f.write(base64.b64decode(pdf_base64))
            doc = pymupdf.open(f"{dump_name}.pdf")
            os.remove(f"{dump_name}.pdf")
            
        all_base64_pdf.append(pdf_base64)

    return all_base64_pdf

# ---------------------------------------------------
# Generate base64 & prompt for PDF parsing
# If these are insurance forms, use "ycbt_ocr_only_prompt" 
# ---------------------------------------------------
def add_pdf_content_json(pdf_paths=None, prompt=PDF_prompt, force_validated=True, is_base64=True):

    import uuid

    if type(pdf_paths)==str:
        pdf_paths = [pdf_paths]
        
    if not is_base64:
        all_base64_pdf = add_pdf_content(pdf_paths=pdf_paths, force_validated=force_validated)
    else:
        all_base64_pdf = pdf_paths
        
    if len(all_base64_pdf) > 0:
        messages = [
            {
                "role": "user",
                "content":[
                    {
                        "type": "pdf",
                        "text": f"{all_base64_pdf[0]}",
                        "prompt": prompt
                    }]
            }
        ]
        return messages
        
    else:
        return None


# ---------------------------------------------------
# Extract PDF (file path or base64 --> text & images) by using EraX API
# ---------------------------------------------------
def extract_PDF_EraX_vLLM(file, pdf_base64=False, 
                          run_OCR=True, 
                          tmp_path="./tmp/", 
                          prompt=PDF_prompt,
                          OCR_func=None, erax_url=erax_url, API_key=API_key_3P):
    import os
    os.makedirs(tmp_path, exist_ok=True)

    if pdf_base64:
        dump_name = str(uuid.uuid4())
        file_name = f"{dump_name}.pdf"
        with open(file_name, "wb") as f:
            f.write(base64.b64decode(file))
        new_file =  file_name
    else:
        file_name = file.split("/")[-1]
        new_file = file
    
    doc = pymupdf.open(new_file) # open a document

    all_content = []
    
    for page_index in tqdm(range(len(doc)), total=len(doc)): # iterate over pdf pages

        page_content = {}
        page_content["page"] =  page_index
        
        page = doc[page_index] # get the page
        
        page_content["text"] =  page.get_text().encode("utf8").decode("utf-8")

        image_list = page.get_images()
    
        # print the number of images found on the page
        if image_list:
            print(f"Found {len(image_list)} images on page {page_index}")
        else:
            print("No images found on page", page_index)

        page_img = []
        
        for image_index, img in enumerate(image_list, start=1): # enumerate the image list
            xref = img[0] # get the XREF of the image
            pix = pymupdf.Pixmap(doc, xref) # create a Pixmap
    
            if pix.n - pix.alpha > 3: # CMYK: convert to RGB first
                pix = pymupdf.Pixmap(pymupdf.csRGB, pix)

            image_path = tmp_path+file_name+"_page_%s-image_%s.png" % (page_index, image_index)
            
            pix.save(image_path) # save the image as png

            if run_OCR:
                tmp, _ = OCR_func(image_path, prompt=prompt, erax_url=erax_url, API_key=API_key)
            else:
                tmp = ""
                
            page_img.append({
                "index": image_index,
                "image": Image.open(image_path),
                "text" : "** Ảnh " + str(image_index) + "/trang " + str(page_index+1) + " **\n" + tmp
            })
            os.remove(image_path)
            
            pix = None
        
        page_content["images_text"] =  page_img
        
        all_content.append(page_content)

    full_text = ""
    for content in all_content:
        full_text += "** Trang " + str(content["page"]+1) + " **\n" + process_lr(content["text"]) + "\n"
        for image_text in content["images_text"]:
            full_text += image_text["text"] + "\n"
        full_text += "\n"
        
    return all_content, full_text

# RunPod long run
- Lưu ý cho PDF nhiều trang hay nhiều ảnh, API sẽ trả về IN_PROGRESS
- Dùng /status để check progress và lấy về

In [7]:
def checkStatusLongRun(ocr_result, erax_url_id=erax_url_id, API_key=API_key_3P):
    import time
    
    final_result =  ocr_result.copy()
    while True:
        time.sleep(0.5)
        if type(final_result)==dict:
            if "status" in final_result.keys() and (final_result["status"]=="IN_PROGRESS" or final_result["status"]=="IN_QUEUE"):
                job_id    =  final_result["id"]
                print(f"Check status & result...{job_id}")
                runpod_status_url = f"https://api.runpod.ai/v2/{erax_url_id}/status/{job_id}"
                head = {}
                head["authorization"] = API_key
                final_result = requests.post(runpod_status_url, headers=head, timeout=120).json()
            else:
                break
        else:        
            break
            
    return final_result

# Image captioning w/ list of [images paths or base64]
- Bạn có thể dùng API này để captioning ảnh
- Lưu ý prompt hợp lý theo đúng kiểu văn bản cần parse
- API chỉ chấp nhận tối đa 20 ảnh nhưng bạn nên captioning tối đa 3 ảnh
- API này kỳ vọng bạn truyền vào list các base64 thuần của ảnh
- Lưu ý prefix: API đã thêm "data:image;base64" trước decoded {base64} của ảnh rồi

In [8]:
def API_Image_OCR_EraX_VL_7B_vLLM(image_paths=None,
                                  is_base64=True, 
                                  prompt=default_prompt, 
                                  erax_url_id=erax_url_id, 
                                  API_key=API_key_3P, 
                                  force_scale=True):

    messages = add_img_content(image_paths, is_base64=is_base64, prompt=prompt, force_scale =force_scale)
    
    content = {
        "generation_config": 
        {
            "temperature": float(0.01),
            "top_p": float(0.001),
            "top_k": int(1),
            "repetition_penalty": float(1.1),
            "max_tokens": 32000
        },
        "messages": messages
    }   

    data_to_send ={
        "input": content
    }

    head = {}
    head["authorization"] = API_key

    erax_url = f"https://api.runpod.ai/v2/{erax_url_id}/runsync"   
    res = requests.post(erax_url, headers=head, json=data_to_send, timeout=3600)
    
    error = False
    
    try:
        result =  res.json()["output"]
    except:
        result =  res.json()
        try:
            result = checkStatusLongRun(result, erax_url_id=erax_url_id)["output"]
        except:
            error = True
                
    content["messages"].append({
            "role": "assistant",
            "content": result,
            "error": error
        }
    )
    
    return result, content

# PDF captioning w/ PDF paths or base64
- Bạn có thể dùng API này để parse PDF cả text & ảnh trong đó
- Lưu ý prompt hợp lý theo đúng kiểu văn bản cần parse
- API chỉ chấp nhận 1 PDF tại 1 thời điểm
- API này kỳ vọng bạn truyền vào đường dẫn đến file PDF or list of PDF's base64

In [9]:
def API_PDF_OCR_EraX_VL_7B_vLLM(pdf_paths=None, is_base64=False,
                                      prompt=PDF_prompt, erax_url_id=erax_url_id, API_key=API_key_3P):

    messages = add_pdf_content_json(pdf_paths, prompt=prompt, is_base64=is_base64)
    
    content = {
        "generation_config": 
        {
            "temperature": float(0.01),
            "top_p": float(0.001),
            "top_k": int(1),
            "repetition_penalty": float(1.1),
            "max_tokens": 32000
        },
        "messages": messages
    }   

    data_to_send ={
        "input": content
    }

    head = {}
    head["authorization"] = API_key

    erax_url = f"https://api.runpod.ai/v2/{erax_url_id}/runsync"   
    res = requests.post(erax_url, headers=head, json=data_to_send, timeout=3600)
    
    error = False
    
    try:
        result =  res.json()["output"]
    except:
        result =  res.json()
        try:
            result = checkStatusLongRun(result, erax_url_id=erax_url_id)["output"]
        except:
            error = True
                        
    content["messages"].append({
            "role": "assistant",
            "content": result,
            "error": error
        }
    )
    
    return result, content


In [10]:
def API_PDF_Base64_OCR_EraX_VL_7B_vLLM(pdf_base64=None, prompt=PDF_prompt, erax_url_id=erax_url_id, API_key=API_key_3P):

    messages = add_pdf_content_json(pdf_base64, prompt=prompt, is_base64=True)
    
    content = {
        "generation_config": 
        {
            "temperature": float(0.01),
            "top_p": float(0.001),
            "top_k": int(1),
            "repetition_penalty": float(1.1),
            "max_tokens": 32000
        },
        "messages": messages
    }   

    data_to_send ={
        "input": content
    }

    head = {}
    head["authorization"] = API_key

    erax_url = f"https://api.runpod.ai/v2/{erax_url_id}/runsync"   
    res = requests.post(erax_url, headers=head, json=data_to_send, timeout=3600)
    
    error = False
    
    try:
        result =  res.json()["output"]
    except:
        result =  res.json()
        try:
            result = checkStatusLongRun(result, erax_url_id=erax_url_id)["output"]
        except:
            error = True
                        
    content["messages"].append({
            "role": "assistant",
            "content": result,
            "error": error
        }
    )
    
    return result, content


# Chat with the previous result from EraX
- Bạn có thể hội thoại liên tục với kết quả EraX đã captioning lần trước hoặc đơn giản là chat với QWen2

In [11]:
def API_Chat_OCR_EraX_VL_7B_vLLM(prompt, history=None, erax_url_id=erax_url_id, API_key=API_key_3P):

    if history is not None:
        history["messages"].append({
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text":prompt
                }
            ]
        })
    else:
        history = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text":prompt
                    }
                ]
            }
        ]
        
    content = {
        "generation_config": 
        {
            "temperature": float(0.1),
            "top_p": float(0.1),
            "top_k": int(10),
            "repetition_penalty": float(1.1),
            "max_tokens": 8192
        },
        "messages": history
    }   

    data_to_send ={
        "input": content
    }

    head = {}
    head["authorization"] = API_key
    
    erax_url = f"https://api.runpod.ai/v2/{erax_url_id}/runsync"   
    res = requests.post(erax_url, headers=head, json=data_to_send, timeout=3600)
    
    error = False
    
    try:
        result =  res.json()["output"]
    except:
        result =  res.json()
        try:
            result = checkStatusLongRun(result, erax_url_id=erax_url_id)["output"]
        except:
            error = True
                                
    content["messages"].append({
            "role": "assistant",
            "content": result
    }
    )
    
    return result, content


# PDF captioning ALL pages w/ PDF paths OR Base64
- Bạn có thể dùng API này để parse PDF cả text & ảnh trong đó
- Lưu ý prompt hợp lý theo đúng kiểu văn bản cần parse
- API chỉ chấp nhận 1 PDF tại 1 thời điểm
- API này kỳ vọng bạn truyền vào đường dẫn đến file PDF hoặc Base64

In [12]:
def API_PDF_Full_OCR_EraX_VL_7B_vLLM(pdf_paths=None,
                                       is_base64=False,
                                       prompt=ycbt_ocr_only_prompt, 
                                       pdf_full_prompt=pdf_full_prompt, 
                                       erax_url_id=erax_url_a100_id, 
                                       API_key=API_key_3P):

    def getPDF_text(json_content):
        text = ""
        for data in json_content:
            text += data["text"]
            for img_text in data["images_text"]:
                text += "\n\n" + img_text["text"] 
        return text
    
    print ("Parsing PDF...")
    ocr_result, history = API_PDF_OCR_EraX_VL_7B_vLLM(pdf_paths=pdf_paths, 
                                                is_base64=is_base64,
                                                prompt=prompt,
                                                erax_url_id=erax_url_id, API_key=API_key)
        
    try:
        final_pdf = json_repair.loads(ocr_result)
        final_pdf =  str(getPDF_text(final_pdf["json_content"]).replace("```json", "").replace("```", "").replace("\n\n", "\n").replace("\n\n", "\n"))
    except Exception as E:
        print("ERROR wrong PDF output format!", str(E))
        return ocr_result, None
        
    print ("Summarize result...")
    pdf_full_prompt_to_send =  pdf_full_prompt.replace("ocr_results", final_pdf)
    
    new_prompt =  f"{pdf_full_prompt_to_send}"

    print (new_prompt)
    
    # Chat w/ API to summarize all into 1
    try:
        final_result, history = API_Chat_OCR_EraX_VL_7B_vLLM(new_prompt, 
                                                             history=None, 
                                                             erax_url_id=erax_url_id, API_key=API_key_3P)
    except Exception as E:
        print ("ERROR chatting w/ API: ", str(E))
        return new_prompt, None
        
    # Done
    final_pdf_text =  final_result.replace("```json", "").replace("```", "").replace("\n\n", "\n").replace("\n\n", "\n")  
    try:
        final_pdf_text = json_repair.loads(final_pdf_text)
    except:
        pass
        
    return final_pdf_text, history  


# Captioning multiple images w/ paths OR Base64
- Bạn có thể dùng API này để parse multiple images cả text & ảnh trong đó
- Lưu ý prompt hợp lý theo đúng kiểu văn bản cần parse


In [16]:
def API_Multiple_Images_OCR_EraX_VL_7B_vLLM(image_paths=None,
                                       is_base64=False,
                                       prompt=ycbt_ocr_only_prompt, 
                                       pdf_full_prompt=pdf_full_prompt, 
                                       erax_url_id=erax_url_a100_id, 
                                       API_key=API_key_3P):

    print ("--> Parsing all images...")

    output_text = ""
    for idx, img_path in enumerate(image_paths):
        print (f"- Parsing image...{idx}")
        add_img_content(img_path, )
        
        ocr_result, _ = API_Image_OCR_EraX_VL_7B_vLLM(image_paths=img_path,
                                                      is_base64=is_base64,
                                                      prompt=prompt,
                                                      erax_url_id=erax_url_id, 
                                                      API_key=API_key)
            
        output_text += f"** Nội dung của giấy tờ trong ảnh số {idx+1}**\n" + \
                           ocr_result.replace("```json", "").replace("```", "") +"\n\n"
                
    print ("--> Summarize result...")
    
    pdf_full_prompt_to_send =  pdf_full_prompt.replace("ocr_results", output_text)
    new_prompt =  f"{pdf_full_prompt_to_send}"

    print (new_prompt)
    
    # Chat w/ API to summarize all into 1
    try:
        final_result, history = API_Chat_OCR_EraX_VL_7B_vLLM(new_prompt, 
                                                             history=None, 
                                                             erax_url_id=erax_url_id, API_key=API_key_3P)
    except Exception as E:
        print ("ERROR chatting w/ API: ", str(E))
        return new_prompt, None
        
    # Done
    final_text =  final_result.replace("```json", "").replace("```", "")
    try:
        final_text = json_repair.loads(final_text)
    except:
        pass
        
    return final_text, history


# TESTING

In [ ]:
# Test PDF long
pdf_sample = "../AAA-BHCN/Digins/sample.pdf"

In [ ]:
final_result, history =  API_PDF_Full_OCR_EraX_VL_7B_vLLM(pdf_paths=pdf_sample, 
                                           is_base64=False,
                                           prompt=ycbt_ocr_only_prompt, 
                                           pdf_full_prompt=pdf_full_prompt, 
                                           erax_url_id=erax_url_a100_id, 
                                           API_key=API_key_3P)

In [ ]:
final_result

In [35]:
# Test multiple images
img_path = ["../samples/2.jpg", "../samples/1.png"]

In [36]:
result, history = API_Multiple_Images_OCR_EraX_VL_7B_vLLM(image_paths=img_path, 
                                                      prompt=ycbt_ocr_only_prompt,
                                                      pdf_full_prompt=pdf_full_prompt, 
                                                      erax_url_id=erax_url_a100_id, 
                                                      API_key=API_key_3P)

--> Parsing all images...
- Parsing image...0
- Parsing image...1
--> Summarize result...

Bạn là một chuyên gia bồi thường bảo hiểm xuất sắc.
Bạn được cung cấp danh sách các json là kết quả đã được OCR chính xác từ một (1) hay nhiều ảnh khác nhau.
Các json này là của một hay nhiều phiếu trong một bộ hồ sơ yêu cầu bồi thường bảo hiểm, bao gồm một hay nhiều hoá đơn các loại.

Bạn có 1 nhiệm vụ: phân tích và tổng hợp các jsons được cung cấp này:
- Phân tích kỹ lưỡng tất cả các json OCR được cung cấp, không đưỡ bỏ sót json nào.
- Tổng hợp tất cả các json trên bằng 1 json khác có tính tổng hợp nhưng đầy đủ chi tiết để công ty tiến hành xem xét thủ tục bồi thường chính xác và công bằng với định dạng json dưới đây.
- Không được bỏ qua bất kỳ chi tiết nào về các triệu chứng, các loại thuốc được kê mua, tên bệnh, đề xuất, các phí dịch vụ y tế và các chi phí khác.
- Không được bỏ qua bất kỳ nội dung nào, kể cả các ghi chú, điều kiện, uỷ quyền, cam kết
- Lưu ý phải tóm tắt ngắn gọn sau khi phân 

In [37]:
result

[{'extraction': 'BỆNH VIỆN ĐA KHOA QUỐC TẾ VINMEC TIMES CITY\nVINMEC TIMES CITY INTERNATIONAL GENERAL HOSPITAL',
  'paper': 'BẢNG KÊ THU PHÍ DỊCH VỤ\nLIST OF SERVICES',
  'customer': [{"ID': '221393', 'họ tên bệnh nhân': 'Phạm Minh Anh', 'bác sĩ điều trị chính': 'Thục Thanh Huyền', 'bác sĩ điều trị phụ': 'Phạm Minh Anh'}],": 'tatus',
    'reasons': ['Khám chuyên khoa'],
    'results': [],
    'conclusions': [],
    'insurance company': [],
    'supplier': [],
    'doctors': [],
    'medicines': [],
    'products and services': {'Dịch vụ chung': {'số lượng': '1',
      'đơn giá': '690.000',
      'thành tiền': '690.000'},
     'Dịch vụ Khác': {'số lượng': '1',
      'đơn giá': '690.000',
      'thành tiền': '690.000'},
     'GM.01': {'số lượng': '1', 'đơn giá': '690.000', 'thành tiền': '690.000'},
     'GM.007?': {'số lượng': '1',
      'đơn giá': '690.000',
      'thành tiền': '690.000'}},
    'total amount': '690.000',
    'others': [],
    'summary': 'Bệnh nhân Phạm Minh Anh đã sử dụ

In [ ]:
history

In [ ]:
# Test 1 ảnh
img_path =  "../samples/d2.jpg"

In [ ]:
result, history = API_Image_OCR_EraX_VL_7B_vLLM(image_paths=img_path,
                                  is_base64=False, 
                                  prompt=popular_prompt, 
                                  erax_url_id=erax_url_a100_id,
                                  API_key=API_key_3P, 
                                  force_scale=False)

In [ ]:
result

In [ ]:
final =  json_repair.loads(result)

In [ ]:
final